# Functions

1. encode_text_features
1. split_dataset
1. scale_train_dataset
1. scale_train_dataset
1. perform_dataset_ops
1. save_learned_models
1. load_learned_models

In [1]:
def encode_text_features(encode_decode, data_frame, encoder_isa=None, encoder_mem_type=None):
  # Implement Categorical OneHot encoding for ISA and mem-type
  if encode_decode == 'encode':
    encoder_isa = ce.one_hot.OneHotEncoder(cols=['isa'])
    encoder_mem_type = ce.one_hot.OneHotEncoder(cols=['mem_type'])
    encoder_isa.fit(data_frame, verbose=1)
    df_new1 = encoder_isa.transform(data_frame)
    encoder_mem_type.fit(df_new1, verbose=1)
    df_new = encoder_mem_type.transform(df_new1)
    encoded_data_frame = df_new
  else:
    df_new1 = encoder_isa.transform(data_frame)
    df_new = encoder_mem_type.transform(df_new1)
    encoded_data_frame = df_new
        
  return encoded_data_frame, encoder_isa, encoder_mem_type

In [2]:
def split_dataset(X, y):
  # split into train and test
  #X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, X.index, test_size=0.2, random_state=42)
  # use default random state of np.random, so not providing any value here
  X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, X.index, test_size=0.2, random_state=0)
  return X_train, X_test, y_train, y_test, indices_train, indices_test

In [3]:
def scale_train_dataset(X_train, y_train, input_scaler=None, output_scaler=None):
  # scale X_train input (fit and transform)
  if input_scaler is not None:
    # fit scaler on training dataset
    input_scaler.fit(X_train)
    # transform training dataset
    X_train = input_scaler.transform(X_train)        
        
  # scale y_train input (fit and transform)
  if output_scaler is not None:
    # fit scaler on training dataset
    output_scaler.fit(y_train)
    # transform training dataset
    y_train = output_scaler.transform(y_train)        
        
  return X_train, y_train, input_scaler, output_scaler

In [4]:
def scale_test_dataset(X_test, y_test, input_scaler, output_scaler):
  # scale X_test input (only transform)
  if input_scaler is not None:
    # transform test dataset
    X_test = input_scaler.transform(X_test)
        
  # scale y_train input (only transform)
  if output_scaler is not None:
    # transform test dataset
    y_test = output_scaler.transform(y_test)
        
  return X_test, y_test

In [5]:
def perform_dataset_ops(benchmark, algo, algo_fname):
  print(benchmark, algo, algo_fname)
  df_orig=pd.read_csv(drive_location+algo_fname+".csv")
  #df_orig.dropna(inplace=True)
    
  # Encode text features
  df_encoded, encoder_isa, encoder_mem_type = encode_text_features('encode', df_orig)
  print("df_orig====================>",df_orig)
  print("df_encoded====================>",df_encoded)
  # store learned models in dictory
  dataset = {}
  dataset['encoder_isa'] = encoder_isa
  dataset['encoder_mem_type'] = encoder_mem_type
    
  dataset['df_orig'] = df_orig
  dataset['df_encoded'] = df_encoded

#  for ps_indx, ps in enumerate(df_orig.PS.unique()):
#    print(ps_indx, ps)
#    dataset[ps] = {}
    
#    criteria_df = (df_encoded['PS'] == ps)
    #print(criteria_df)
#    df_selected = df_encoded[criteria_df]
  df_selected = df_encoded    
  if (benchmark == "SPEC2006" or benchmark == "SPEC2017"):
    X_all=df_selected.drop(columns=['arch','ld_shared_by_cores','l2_shared_by_cores','no_of_threads','system_name','bus_speed_qpi','bus_speed_dmi','l1_ins_assoc','l1_data_assoc','l2_assoc','l3_assoc','raw_bus_speed','converted_bus_speed','ddr_type','runtime'])
  elif (benchmark == "NPB"):
    X_all=df_selected.drop(columns=['sys','sysname','arch','l1d_assoc','l1d_cache_lines','l1d_shared_by_threads','l2_assoc','l2_cache_lines','l2_shared_by_threads','l3_assoc','l3_cache_lines','l3_shared_by_threads','PS','runtime','executable','system'])
  else:
    X_all=df_selected.drop(columns=['sys','arch','l1d_assoc','l1d_cache_lines','l1d_shared_by_threads','l2_assoc','l2_cache_lines','l2_shared_by_threads','l3_assoc','l3_cache_lines','l3_shared_by_threads','runtime'])
  #print(X_all)
  X_all.dropna(inplace=True)
  dataset['X_all'] = X_all
  
  # Output
  y=df_selected[['runtime']]
  print("y shape ",y.shape, "y types", y.dtypes)
  #print("y shape ",y.shape, "y types", y.dtypes)
  dataset['y'] = y

  # split into train and test
  X_train, X_test, y_train, y_test, indices_train, indices_test = split_dataset(X_all, y)
  dataset['X_train'] = X_train
  dataset['X_test'] = X_test
  dataset['y_train'] = y_train
  dataset['y_test'] = y_test
  dataset['indices_train'] = indices_train
  dataset['indices_test'] = indices_test

  # unscaled inputs
  # scale train data (fit and transform)
  X_train_unscaled, y_train_unscaled, input_scaler_unscaled, output_scaler_unscaled = scale_train_dataset(X_train, y_train, None, StandardScaler())
  # scale test data (only transform)
  X_test_unscaled, y_test_unscaled = scale_test_dataset(X_test, y_test, input_scaler_unscaled, output_scaler_unscaled)
  dataset['X_train_unscaled'] = X_train_unscaled
  dataset['X_test_unscaled'] = X_test_unscaled
  dataset['y_train_unscaled'] = y_train_unscaled
  dataset['y_test_unscaled'] = y_test_unscaled
  dataset['input_scaler_unscaled'] = input_scaler_unscaled
  dataset['output_scaler_unscaled'] = output_scaler_unscaled
      
  #standardized inputs
  # scale train data (fit and transform)
  X_train_standardized, y_train_standardized, input_scaler_standardized, output_scaler_standardized = scale_train_dataset(X_train, y_train, StandardScaler(), StandardScaler())
  # scale test data (only transform)
  X_test_standardized, y_test_standardized = scale_test_dataset(X_test, y_test, input_scaler_standardized, output_scaler_standardized)
  dataset['X_train_standardized'] = X_train_standardized
  dataset['X_test_standardized'] = X_test_standardized
  dataset['y_train_standardized'] = y_train_standardized
  dataset['y_test_standardized'] = y_test_standardized
  dataset['input_scaler_standardized'] = input_scaler_standardized
  dataset['output_scaler_standardized'] = output_scaler_standardized
      
  save_learned_models(dataset, algo, "dataset")

In [6]:
def save_learned_models(dct, algo, fname):
  pickleFile = drive_location+algo+"_"+fname+".pickle"
  with open(pickleFile, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(dct, f, pickle.HIGHEST_PROTOCOL)

In [7]:
def save_dict_to_csv(dct,fname,name='algos'):
  fileName =  drive_location+fname+".csv"
  header = sorted(set(i for b in map(dict.keys, dct.values()) for i in b))
  with open(fileName, 'w', newline="") as f:
    write = csv.writer(f)
    write.writerow([name, *header])
    for a, b in dct.items():
      write.writerow([a]+[b.get(i, '') for i in header])

In [8]:
def load_learned_models(algo, fname):
  pickleFile = drive_location+algo+"_"+fname+".pickle"
  f = open(pickleFile, 'rb')
  # disable garbage collector
  gc.disable()
  # The protocol version used is detected automatically, so we do not
  # have to specify it.
  data = pickle.load(f)
  # enable garbage collector again
  gc.enable()
  f.close()
  return data

# model

1. nn model
1. keras model

## NN model

In [9]:
def build_nn_model(algo):
  dataset = load_learned_models(algo, "dataset")
  nnmodel = {}
  input_dim = dataset['X_train_standardized'].shape[1]
  output_dim = dataset['y_train_standardized'].shape[1]
  print('input_dim:', input_dim, 'output_dim:', output_dim)
  model_iter = 0
  rmse = 9999999999999999999999999
  model = tf.keras.Sequential()
  model.add(tf.keras.Input(shape=(input_dim,)))
  while (True):
    if (model_iter > 0):
      # add layer with neurons = rmse*100
      model.add(tf.keras.layers.Dense(int(rmse*100), activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(output_dim, activation='linear'))
    # compile model
    model.compile(loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer='adam')
    # fit model
    model.fit(dataset['X_train_standardized'], dataset['y_train_standardized'], epochs=100, verbose=0)
    # evaluate the model
    y_train_pred = model.predict(dataset['X_train_standardized'])
    y_test_pred = model.predict(dataset['X_test_standardized'])
    prev_rmse = rmse
    rmse = mean_squared_error(dataset['y_train_standardized'],y_train_pred,squared=False)
    print(rmse, prev_rmse)
    if (rmse > prev_rmse):
      break
    #nnmodel['model'] = model
    model.save(drive_location+algo+"_nnmodel.h5")
    nnmodel[model_iter] = {}
    nnmodel[model_iter]['y_train'] = dataset['y_train_standardized']
    nnmodel[model_iter]['y_train_pred'] = y_train_pred
    nnmodel[model_iter]['y_test'] = dataset['y_test_standardized']
    nnmodel[model_iter]['y_test_pred'] = y_test_pred
    nnmodel[model_iter]['train_rmse'] = mean_squared_error(dataset['y_train_standardized'],y_train_pred,squared=False)
    nnmodel[model_iter]['train_medae'] = median_absolute_error(dataset['y_train_standardized'],y_train_pred)
    nnmodel[model_iter]['train_explained_var'] = explained_variance_score(dataset['y_train_standardized'],y_train_pred)
    nnmodel[model_iter]['test_rmse'] = mean_squared_error(dataset['y_test_standardized'],y_test_pred,squared=False)
    nnmodel[model_iter]['test_medae'] = median_absolute_error(dataset['y_test_standardized'],y_test_pred)
    nnmodel[model_iter]['test_explained_var'] = explained_variance_score(dataset['y_test_standardized'],y_test_pred)
    nnmodel[model_iter]['y_train_pred_error'] = np.abs((dataset['output_scaler_standardized'].inverse_transform(dataset['y_train_standardized']) - dataset['output_scaler_standardized'].inverse_transform(y_train_pred))/dataset['output_scaler_standardized'].inverse_transform(dataset['y_train_standardized'])*100)
    nnmodel[model_iter]['y_test_pred_error'] = np.abs((dataset['output_scaler_standardized'].inverse_transform(dataset['y_test_standardized']) - dataset['output_scaler_standardized'].inverse_transform(y_test_pred))/dataset['output_scaler_standardized'].inverse_transform(dataset['y_test_standardized'])*100)
    nnmodel[model_iter]['train_medape'] = np.median(nnmodel[model_iter]['y_train_pred_error'])
    nnmodel[model_iter]['test_medape'] = np.median(nnmodel[model_iter]['y_test_pred_error'])
    
    print('Neural Network Iteration {0}'.format(model_iter))
    print('Training Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(nnmodel[model_iter]['train_rmse'], nnmodel[model_iter]['train_medae'], nnmodel[model_iter]['train_explained_var'], nnmodel[model_iter]['train_medape']))
    print('Testing Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(nnmodel[model_iter]['test_rmse'], nnmodel[model_iter]['test_medae'], nnmodel[model_iter]['test_explained_var'], nnmodel[model_iter]['test_medape']))
    print(model.summary())
    model_iter = model_iter + 1
    # remove the output layer from previously built model
    model.pop()


  save_learned_models(nnmodel, algo, "nnmodel")
  print('Final Model--->')
  model1 = tf.keras.models.load_model(drive_location+algo+"_nnmodel.h5")
  model1.summary()


## keras model

In [10]:
def build_autokeras(algo):
  dataset = load_learned_models(algo, "dataset")
  reg = ak.StructuredDataRegressor(max_trials=3, overwrite=True)
  # Feed the tensorflow Dataset to the regressor.
  reg.fit(dataset['X_train_standardized'], dataset['y_train_standardized'], epochs=100)
  # Predict with the best model.
  y_train_pred = reg.predict(dataset['X_train_standardized'])
  y_test_pred = reg.predict(dataset['X_test_standardized'])

  autokerasmodel = {}
  autokerasmodel['y_train'] = dataset['y_train_standardized']
  autokerasmodel['y_train_pred'] = y_train_pred
  autokerasmodel['y_test'] = dataset['y_test_standardized']
  autokerasmodel['y_test_pred'] = y_test_pred
  autokerasmodel['train_rmse'] = mean_squared_error(dataset['y_train_standardized'],y_train_pred,squared=False)
  autokerasmodel['train_medae'] = median_absolute_error(dataset['y_train_standardized'],y_train_pred)
  autokerasmodel['train_explained_var'] = explained_variance_score(dataset['y_train_standardized'],y_train_pred)
  autokerasmodel['test_rmse'] = mean_squared_error(dataset['y_test_standardized'],y_test_pred,squared=False)
  autokerasmodel['test_medae'] = median_absolute_error(dataset['y_test_standardized'],y_test_pred)
  autokerasmodel['test_explained_var'] = explained_variance_score(dataset['y_test_standardized'],y_test_pred)
  autokerasmodel['y_train_pred_error'] = np.abs((dataset['output_scaler_standardized'].inverse_transform(dataset['y_train_standardized']) - dataset['output_scaler_standardized'].inverse_transform(y_train_pred))/dataset['output_scaler_standardized'].inverse_transform(dataset['y_train_standardized'])*100)
  autokerasmodel['y_test_pred_error'] = np.abs((dataset['output_scaler_standardized'].inverse_transform(dataset['y_test_standardized']) - dataset['output_scaler_standardized'].inverse_transform(y_test_pred))/dataset['output_scaler_standardized'].inverse_transform(dataset['y_test_standardized'])*100)
  autokerasmodel['train_medape'] = np.median(autokerasmodel['y_train_pred_error'])
  autokerasmodel['test_medape'] = np.median(autokerasmodel['y_test_pred_error'])
    
  print('Auto Keras-->')
  print('Training Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(autokerasmodel['train_rmse'], autokerasmodel['train_medae'], autokerasmodel['train_explained_var'], autokerasmodel['train_medape']))
  print('Testing Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(autokerasmodel['test_rmse'], autokerasmodel['test_medae'], autokerasmodel['test_explained_var'], autokerasmodel['test_medape']))

  # Export as a Keras Model.
  model = reg.export_model()
  try:
    model.save(drive_location+algo+"_model_autokeras", save_format="tf")
    loaded_model = load_model(drive_location+algo+"_model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)
    #loaded_model = load_model(drive_location+algo+"_model_autokeras")
    print(loaded_model.predict(tf.expand_dims(dataset['X_test_standardized'], -1)))
  except Exception:
    model.save(drive_location+algo+"_autokeras.h5")

  save_learned_models(autokerasmodel, algo, "autokeras")

## autoPytorch

- serch 
  - xtrain
  - ytrain
  - optimize metrix
    - 'mean_absolute_error'
    - 'mean_squared_error'
    - 'root_mean_squared_error'
    - 'mean_squared_log_error'
    - 'median_absolute_error'
    - 'r2' (default)

In [11]:
def build_autoPytorch(algo,optimizeMetric='r2'):
  dataset = load_learned_models(algo,"dataset")
  reg = TabularRegressionTask()
  reg.search(X_train=dataset['X_train_standardized'],
             y_train=dataset['y_train_standardized'],
             optimize_metric=optimizeMetric, total_walltime_limit=300,func_eval_time_limit_secs=50)
  y_train_pred = reg.predict(dataset['X_train_standardized'])
  y_test_pred = reg.predict(dataset['X_test_standardized'])

  score1 = reg.score(y_train_pred,dataset['y_train_standardized']);
  score2 = reg.score(y_test_pred,dataset['y_test_standardized']);
  print("before ============>",score1);
  print("before ============>",score2);
 
  autoputorchmodel = {}
  autoputorchmodel['y_train'] = dataset['y_train_standardized']
  autoputorchmodel['y_train_pred'] = y_train_pred
  autoputorchmodel['y_test'] = dataset['y_test_standardized']
  autoputorchmodel['y_test_pred'] = y_test_pred
  autoputorchmodel['train_rmse'] = mean_squared_error(dataset['y_train_standardized'],y_train_pred,squared=False)
  autoputorchmodel['train_medae'] = median_absolute_error(dataset['y_train_standardized'],y_train_pred)
  autoputorchmodel['train_explained_var'] = explained_variance_score(dataset['y_train_standardized'],y_train_pred)
  autoputorchmodel['test_rmse'] = mean_squared_error(dataset['y_test_standardized'],y_test_pred,squared=False)
  autoputorchmodel['test_medae'] = median_absolute_error(dataset['y_test_standardized'],y_test_pred)
  autoputorchmodel['test_explained_var'] = explained_variance_score(dataset['y_test_standardized'],y_test_pred)
  autoputorchmodel['y_train_pred_error'] = np.abs((dataset['output_scaler_standardized'].inverse_transform(dataset['y_train_standardized']) - dataset['output_scaler_standardized'].inverse_transform(y_train_pred))/dataset['output_scaler_standardized'].inverse_transform(dataset['y_train_standardized'])*100)
  autoputorchmodel['y_test_pred_error'] = np.abs((dataset['output_scaler_standardized'].inverse_transform(dataset['y_test_standardized']) - dataset['output_scaler_standardized'].inverse_transform(y_test_pred))/dataset['output_scaler_standardized'].inverse_transform(dataset['y_test_standardized'])*100)
  autoputorchmodel['train_medape'] = np.median(autoputorchmodel['y_train_pred_error'])
  autoputorchmodel['test_medape'] = np.median(autoputorchmodel['y_test_pred_error'])
    
  print('autopytorch-->')
  print('Training Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(autoputorchmodel['train_rmse'], autoputorchmodel['train_medae'], autoputorchmodel['train_explained_var'], autoputorchmodel['train_medape']))
  print('Testing Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(autoputorchmodel['test_rmse'], autoputorchmodel['test_medae'], autoputorchmodel['test_explained_var'], autoputorchmodel['test_medape']))

  torch.save(reg,drive_location+algo+optimizeMetric+"_model_autopytorch")
  loaded_model = torch.load(drive_location+algo+optimizeMetric+"_model_autopytorch")
  #loaded_model = load_model(drive_location+algo+"_model_autokeras")
  y_train_pred_after = loaded_model.predict(dataset['X_train_standardized'])
  y_test_pred_after = loaded_model.predict(dataset['X_test_standardized'])

  score1 = loaded_model.score(y_train_pred_after,dataset['y_train_standardized']);
  score2 = loaded_model.score(y_test_pred_after,dataset['y_test_standardized']);
  print("after ============>",score1);
  print("after ============>",score2);

  save_learned_models(autoputorchmodel, algo+optimizeMetric, "autopytorch")


## Mljar

AutoML will use algorithms: ['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']


In [12]:
def build_mljar(algo):
  dataset = load_learned_models(algo,"dataset")
  automl = AutoML(total_time_limit=60*60,results_path=drive_location+'mljar/'+algo) # 1 hour
  print(dataset['y_train_standardized'].flatten())
  automl.fit(dataset['X_train_standardized'],dataset['y_train_standardized'].flatten())

  y_train_pred = automl.predict_all(dataset['X_train_standardized'])
  y_test_pred = automl.predict_all(dataset['X_test_standardized'])

  print(y_test_pred)
  print(dataset['y_test_standardized'])
  
  autoMlJarModel = {}
  autoMlJarModel['y_train'] = dataset['y_train_standardized']
  autoMlJarModel['y_train_pred'] = y_train_pred
  autoMlJarModel['y_test'] = dataset['y_test_standardized']
  autoMlJarModel['y_test_pred'] = y_test_pred
  autoMlJarModel['train_rmse'] = mean_squared_error(dataset['y_train_standardized'],y_train_pred,squared=False)
  autoMlJarModel['train_medae'] = median_absolute_error(dataset['y_train_standardized'],y_train_pred)
  autoMlJarModel['train_explained_var'] = explained_variance_score(dataset['y_train_standardized'],y_train_pred)
  autoMlJarModel['test_rmse'] = mean_squared_error(dataset['y_test_standardized'],y_test_pred,squared=False)
  autoMlJarModel['test_medae'] = median_absolute_error(dataset['y_test_standardized'],y_test_pred)
  autoMlJarModel['test_explained_var'] = explained_variance_score(dataset['y_test_standardized'],y_test_pred)
  autoMlJarModel['y_train_pred_error'] = np.abs((dataset['output_scaler_standardized'].inverse_transform(dataset['y_train_standardized']) - dataset['output_scaler_standardized'].inverse_transform(y_train_pred))/dataset['output_scaler_standardized'].inverse_transform(dataset['y_train_standardized'])*100)
  autoMlJarModel['y_test_pred_error'] = np.abs((dataset['output_scaler_standardized'].inverse_transform(dataset['y_test_standardized']) - dataset['output_scaler_standardized'].inverse_transform(y_test_pred))/dataset['output_scaler_standardized'].inverse_transform(dataset['y_test_standardized'])*100)
  autoMlJarModel['train_medape'] = np.median(autoMlJarModel['y_train_pred_error'])
  autoMlJarModel['test_medape'] = np.median(autoMlJarModel['y_test_pred_error'])
    
  print('Auto Mljar-->')
  print('Training Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(autoMlJarModel['train_rmse'], autoMlJarModel['train_medae'], autoMlJarModel['train_explained_var'], autoMlJarModel['train_medape']))
  print('Testing Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(autoMlJarModel['test_rmse'], autoMlJarModel['test_medae'], autoMlJarModel['test_explained_var'], autoMlJarModel['test_medape']))
  
  save_learned_models(autoMlJarModel, algo, "automljar")

# plotting Functions

1. set_pub
1. capture_metric_in_csv
1. plot_nnmodel_vs_autokeras
1. capture_iteration_metric_in_csv
1. plot_iteration_metric


In [13]:
def set_pub():
    mpl.rcdefaults()
    mpl.rcParams['font.size'] = 14
    #mpl.rcParams['font.weight'] = 'bold'
    mpl.rcParams['xtick.labelsize'] = 'xx-large'
    mpl.rcParams['ytick.labelsize'] = 'xx-large'
    mpl.rcParams['figure.titlesize'] = 'xx-large'
    mpl.rcParams['legend.fontsize'] = 'xx-large'
    mpl.rcParams['axes.titlesize'] = 'xx-large'
    mpl.rcParams['axes.titleweight'] = 'bold'
    mpl.rcParams['axes.labelsize'] = 'xx-large'
    mpl.rcParams['axes.labelweight'] = 'bold'
    mpl.rcParams['lines.linewidth'] = 3
    mpl.rcParams['lines.markersize'] = 12
    print(mpl.rcParams['figure.titlesize'], mpl.rcParams['legend.fontsize'])
    plt.rc('figure', dpi=500)

## nnVs keras comparision

In [14]:
def capture_metric_in_csv(benchmark_title, algo_list):
  # First capture data in csv file for all algo
  metric_list = []
  for algo in algo_list:
    nnmodel = load_learned_models(algo, "nnmodel")
    final_nnmodel_key = list(nnmodel.keys())[-1]
    autokerasmodel = load_learned_models(algo, "autokeras")
    metric_dict = {}
    metric_dict['algo'] = algo
    # RMSE
    metric_dict['nnmodel_train_rmse'] = nnmodel[final_nnmodel_key]['train_rmse']
    metric_dict['autokerasmodel_train_rmse'] = autokerasmodel['train_rmse']
    metric_dict['nnmodel_test_rmse'] = nnmodel[final_nnmodel_key]['test_rmse']
    metric_dict['autokerasmodel_test_rmse'] = autokerasmodel['test_rmse']
    # MedAE
    metric_dict['nnmodel_train_medae'] = nnmodel[final_nnmodel_key]['train_medae']
    metric_dict['autokerasmodel_train_medae'] = autokerasmodel['train_medae']
    metric_dict['nnmodel_test_medae'] = nnmodel[final_nnmodel_key]['test_medae']
    metric_dict['autokerasmodel_test_medae'] = autokerasmodel['test_medae']
    # MedAPE
    metric_dict['nnmodel_train_medape'] = nnmodel[final_nnmodel_key]['train_medape']
    metric_dict['autokerasmodel_train_medape'] = autokerasmodel['train_medape']
    metric_dict['nnmodel_test_medape'] = nnmodel[final_nnmodel_key]['test_medape']
    metric_dict['autokerasmodel_test_medape'] = autokerasmodel['test_medape']

    print(metric_dict)
    metric_list.append(metric_dict)
  metric_df = pd.DataFrame.from_dict(metric_list)
  metric_df.to_csv(drive_location+benchmark_title+"_metric.csv", line_terminator='\n', index=False)

In [15]:
def plot_nnmodel_vs_autokeras(benchmark_title, algo_list):
  capture_metric_in_csv(benchmark_title, algo_list)

  metric_df = pd.read_csv(drive_location+benchmark_title+"_metric.csv")
  y_pos = np.arange(metric_df.shape[0])
  print(y_pos)


  # Plot Train and Test MedAE for nnmodel (our model) and autokeras model 
  colors = ['r','k','b','y']
  fig, axes = plt.subplots(figsize=(len(y_pos)*1.5, 4))
  rect1 = []
  rect2 = []
  rect3 = []
  rect4 = []
  width = 0.2
  rect1 = plt.bar(y_pos, metric_df['nnmodel_train_medae'], width, color=colors[0])
  rect2 = plt.bar(y_pos+width, metric_df['autokerasmodel_train_medae'], width, color=colors[1])
  rect3 = plt.bar(y_pos+2*width, metric_df['nnmodel_test_medae'], width, color=colors[2])
  rect4 = plt.bar(y_pos+3*width, metric_df['autokerasmodel_test_medae'], width, color=colors[3])
  
  plt.xticks(y_pos+1.5*width, metric_df['algo'], rotation=20, ha='right')
  plt.ylabel('MedAE')
  plt.xlabel(benchmark_title)
  #lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  lgd = plt.legend((rect1, rect2, rect3,rect4), ("Our Model Train MedAE", "Autokeras Model Train MedAE", "Our Model Test MedAE", "Autokeras Test MedAE"), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()
  strFile = drive_location + benchmark_title + "_nnmodel_vs_autokerasmodel_medae"+plot_file_ext
  if os.path.isfile(strFile):
    os.remove(strFile)
  #fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
  fig.savefig(strFile, bbox_inches='tight')


  # Plot Train and Test MedAPE for nnmodel (our model) and autokeras model 
  colors = ['r','k','b','y']
  fig, axes = plt.subplots(figsize=(len(y_pos)*1.5, 4))
  rect1 = []
  rect2 = []
  rect3 = []
  rect4 = []
  width = 0.2
  rect1 = plt.bar(y_pos, metric_df['nnmodel_train_medape'], width, color=colors[0])
  rect2 = plt.bar(y_pos+width, metric_df['autokerasmodel_train_medape'], width, color=colors[1])
  rect3 = plt.bar(y_pos+2*width, metric_df['nnmodel_test_medape'], width, color=colors[2])
  rect4 = plt.bar(y_pos+3*width, metric_df['autokerasmodel_test_medape'], width, color=colors[3])
  
  plt.xticks(y_pos+1.5*width, metric_df['algo'], rotation=20, ha='right')
  plt.ylabel('MedAPE')
  plt.xlabel(benchmark_title)
  #lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  lgd = plt.legend((rect1, rect2, rect3,rect4), ("Our Model Train MedAPE", "Autokeras Model Train MedAPE", "Our Model Test MedAPE", "Autokeras Test MedAPE"), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()
  strFile = drive_location + benchmark_title + "_nnmodel_vs_autokerasmodel_medape"+plot_file_ext
  if os.path.isfile(strFile):
    os.remove(strFile)
  #fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
  fig.savefig(strFile, bbox_inches='tight')


  # drop RMSE values for SPEC2006 Int Benchmarks due to very high value
  if (benchmark_title == 'SPEC_Int'):
    indexNames = metric_df[metric_df['algo'].isin(['401.bzip2', '456.hmmer', '458.sjeng', '462.libquantum', '473.astar'])].index
    #print(indexNames)
    metric_df = metric_df.drop(indexNames)
    #print(metric_df)
  y_pos = np.arange(metric_df.shape[0])
  print(y_pos)

   # Plot Train and Test RMSE for nnmodel (our model) and autokeras model 
  colors = ['r','k','b','y']
  fig, axes = plt.subplots(figsize=(len(y_pos)*1.5, 4))
  rect1 = []
  rect2 = []
  rect3 = []
  rect4 = []
  width = 0.2
  rect1 = plt.bar(y_pos, metric_df['nnmodel_train_rmse'], width, color=colors[0])
  rect2 = plt.bar(y_pos+width, metric_df['autokerasmodel_train_rmse'], width, color=colors[1])
  rect3 = plt.bar(y_pos+2*width, metric_df['nnmodel_test_rmse'], width, color=colors[2])
  rect4 = plt.bar(y_pos+3*width, metric_df['autokerasmodel_test_rmse'], width, color=colors[3])
  
  plt.xticks(y_pos+1.5*width, metric_df['algo'], rotation=20, ha='right')
  plt.ylabel('RMSE')
  plt.xlabel(benchmark_title)
  #lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  lgd = plt.legend((rect1, rect2, rect3,rect4), ("Our Model Train RMSE", "Autokeras Model Train RMSE", "Our Model Test RMSE", "Autokeras Test RMSE"), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()
  strFile = drive_location + benchmark_title + "_nnmodel_vs_autokerasmodel_rmse"+plot_file_ext
  if os.path.isfile(strFile):
    os.remove(strFile)
  #fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
  fig.savefig(strFile, bbox_inches='tight')


## comparision of all metric of pytorch

In [16]:
def capture_metric_in_csv_all_pytorch(benchmark_title, algo_list):
  # First capture data in csv file for all algo
  metric_list = []
  for algo in algo_list:
    mean_absolute_error = load_learned_models(algo+"mean_absolute_error", "autopytorch")
    mean_squared_error = load_learned_models(algo+"mean_squared_error", "autopytorch")
    root_mean_squared_error = load_learned_models(algo + "root_mean_squared_error", "autopytorch")
    median_absolute_error = load_learned_models(algo + "median_absolute_error", "autopytorch")
    r2 = load_learned_models(algo + "r2", "autopytorch")
    metric_dict = {}
    metric_dict['algo'] = algo
    # RMSE
    metric_dict['mean_absolute_error_train_rmse'] = mean_absolute_error['train_rmse']
    metric_dict['mean_squared_error_train_rmse'] = mean_squared_error['train_rmse']
    metric_dict['root_mean_squared_error_train_rmse'] = root_mean_squared_error['train_rmse']
    metric_dict['median_absolute_error_train_rmse'] = median_absolute_error['train_rmse']
    metric_dict['r2_train_rmse'] = r2['train_rmse']

    metric_dict['mean_absolute_error_test_rmse'] = mean_absolute_error['test_rmse']
    metric_dict['mean_squared_error_test_rmse'] = mean_squared_error['test_rmse']
    metric_dict['root_mean_squared_error_test_rmse'] = root_mean_squared_error['test_rmse']
    metric_dict['median_absolute_error_test_rmse'] = median_absolute_error['test_rmse']
    metric_dict['r2_test_rmse'] = r2['test_rmse']
    # MedAE
    metric_dict['mean_absolute_error_train_medae'] = mean_absolute_error['train_medae']
    metric_dict['mean_squared_error_train_medae'] = mean_squared_error['train_medae']
    metric_dict['root_mean_squared_error_train_medae'] = root_mean_squared_error['train_medae']
    metric_dict['median_absolute_error_train_medae'] = median_absolute_error['train_medae']
    metric_dict['r2_train_medae'] = r2['train_medae']

    metric_dict['mean_absolute_error_test_medae'] = mean_absolute_error['test_medae']
    metric_dict['mean_squared_error_test_medae'] = mean_squared_error['test_medae']
    metric_dict['root_mean_squared_error_test_medae'] = root_mean_squared_error['test_medae']
    metric_dict['median_absolute_error_test_medae'] = median_absolute_error['test_medae']
    metric_dict['r2_test_medae'] = r2['test_medae']

    # MedAPE
    metric_dict['mean_absolute_error_train_medape'] = mean_absolute_error['train_medape']
    metric_dict['mean_squared_error_train_medape'] = mean_squared_error['train_medape']
    metric_dict['root_mean_squared_error_train_medape'] = root_mean_squared_error['train_medape']
    metric_dict['median_absolute_error_train_medape'] = median_absolute_error['train_medape']
    metric_dict['r2_train_medape'] = r2['train_medape']

    metric_dict['mean_absolute_error_test_medape'] = mean_absolute_error['test_medape']
    metric_dict['mean_squared_error_test_medape'] = mean_squared_error['test_medape']
    metric_dict['root_mean_squared_error_test_medape'] = root_mean_squared_error['test_medape']
    metric_dict['median_absolute_error_test_medape'] = median_absolute_error['test_medape']
    metric_dict['r2_test_medape'] = r2['test_medape']

    print(metric_dict)
    metric_list.append(metric_dict)
  metric_df = pd.DataFrame.from_dict(metric_list)
  metric_df.to_csv(drive_location+benchmark_title+"allAlgo"+"_metric.csv", line_terminator='\n', index=False)


In [17]:
def plot_all_pytorch(benchmark_title, algo_list):
    capture_metric_in_csv_all_pytorch(benchmark_title, algo_list)

    metric_df = pd.read_csv(drive_location + benchmark_title + "allAlgo" + "_metric.csv")
    y_pos = np.arange(metric_df.shape[0])
    print(y_pos)

    colors = ['r','k','b','y','g','r','k','b','y','g']
    fig, axes = plt.subplots(figsize=(len(y_pos) * 2, 4))
    rect1 = []
    rect2 = []
    rect3 = []
    rect4 = []
    rect5 = []
    rect5 = []
    rect7 = []
    rect8 = []
    rect9 = []
    rect10 = []
    width = 0.09
    rect1 = plt.bar(y_pos            , metric_df['mean_absolute_error_train_medae'], width, color=colors[0])
    rect2 = plt.bar(y_pos +     width, metric_df['mean_squared_error_train_medae'], width, color=colors[1])
    rect3 = plt.bar(y_pos + 2 * width, metric_df['root_mean_squared_error_train_medae'], width, color=colors[2])
    rect4 = plt.bar(y_pos + 3 * width, metric_df['median_absolute_error_train_medae'], width, color=colors[3])
    rect5 = plt.bar(y_pos + 4 * width, metric_df['r2_train_medae'], width, color=colors[4])
    rect6 = plt.bar(y_pos + 5 * width, metric_df['mean_absolute_error_test_medae'], width, color=colors[5])
    rect7 = plt.bar(y_pos + 6 * width, metric_df['mean_squared_error_test_medae'], width, color=colors[6])
    rect8 = plt.bar(y_pos + 7 * width, metric_df['root_mean_squared_error_test_medae'], width, color=colors[7])
    rect9 = plt.bar(y_pos + 8 * width, metric_df['median_absolute_error_test_medae'], width, color=colors[8])
    rect10 = plt.bar(y_pos + 9 * width, metric_df['r2_test_medae'], width, color=colors[9])

    plt.xticks(y_pos + 1 * width, metric_df['algo'], rotation=20, ha='right')
    plt.ylabel('MedAE')
    plt.xlabel(benchmark_title)
    # lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    lgd = plt.legend((rect1, rect2, rect3, rect4, rect5, rect6, rect7, rect8, rect9, rect10),
                     ("mean_absolute_error train MedAE", "mean_squared_error train MedAE",
                      "root_mean_squared_error train MedAE", "median_absolute_error train MedAE", "r2 train MedAE"
                      , "mean_absolute_error test MedAE", "mean_squared_error test MedAE",
                      "root_mean_squared_error test MedAE", "median_absolute_error test MedAE", "r2 test MedAE"),
                     bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    strFile = drive_location + benchmark_title + "_AllMetrix_MedAE" + plot_file_ext
    if os.path.isfile(strFile):
        os.remove(strFile)
    # fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(strFile, bbox_inches='tight')

    # Plot Train and Test MedAPE for nnmodel (our model) and autokeras model
    colors = ['r','k','b','y','g','r','k','b','y','g']
    fig, axes = plt.subplots(figsize=(len(y_pos) * 2, 4))
    rect1 = []
    rect2 = []
    rect3 = []
    rect4 = []
    rect5 = []
    rect5 = []
    rect7 = []
    rect8 = []
    rect9 = []
    rect10 = []
    width = 0.09
    rect1 = plt.bar(y_pos, metric_df['mean_absolute_error_train_medape'], width, color=colors[0])
    rect2 = plt.bar(y_pos + width, metric_df['mean_squared_error_train_medape'], width, color=colors[1])
    rect3 = plt.bar(y_pos + 2 * width, metric_df['root_mean_squared_error_train_medape'], width, color=colors[2])
    rect4 = plt.bar(y_pos + 3 * width, metric_df['median_absolute_error_train_medape'], width, color=colors[3])
    rect5 = plt.bar(y_pos + 4 * width, metric_df['r2_train_medape'], width, color=colors[4])
    rect6 = plt.bar(y_pos + 5 * width, metric_df['mean_absolute_error_test_medape'], width, color=colors[5])
    rect7 = plt.bar(y_pos + 6 * width, metric_df['mean_squared_error_test_medape'], width, color=colors[6])
    rect8 = plt.bar(y_pos + 7 * width, metric_df['root_mean_squared_error_test_medape'], width, color=colors[7])
    rect9 = plt.bar(y_pos + 8 * width, metric_df['median_absolute_error_test_medape'], width, color=colors[8])
    rect10 = plt.bar(y_pos + 9 * width, metric_df['r2_test_medape'], width, color=colors[9])

    plt.xticks(y_pos + 1.5 * width, metric_df['algo'], rotation=20, ha='right')
    plt.ylabel('MedAPE')
    plt.xlabel(benchmark_title)
    # lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    lgd = plt.legend((rect1, rect2, rect3, rect4, rect5, rect6, rect7, rect8, rect9, rect10),
                     ("mean_absolute_error train MedAE", "mean_squared_error train MedAE",
                      "root_mean_squared_error train MedAE", "median_absolute_error train MedAE", "r2 train MedAE"
                      , "mean_absolute_error test MedAE", "mean_squared_error test MedAE",
                      "root_mean_squared_error test MedAE", "median_absolute_error test MedAE", "r2 test MedAE"),
                     bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    strFile = drive_location + benchmark_title + "_AllMetrix_medape" + plot_file_ext
    if os.path.isfile(strFile):
        os.remove(strFile)
    # fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(strFile, bbox_inches='tight')

    # drop RMSE values for SPEC2006 Int Benchmarks due to very high value
    if (benchmark_title == 'SPEC_Int'):
        indexNames = metric_df[
            metric_df['algo'].isin(['401.bzip2', '456.hmmer', '458.sjeng', '462.libquantum', '473.astar'])].index
        # print(indexNames)
        metric_df = metric_df.drop(indexNames)
        # print(metric_df)
    y_pos = np.arange(metric_df.shape[0])
    print(y_pos)

    # Plot Train and Test RMSE for nnmodel (our model) and autokeras model
    colors = ['r','k','b','y','g','r','k','b','y','g']
    fig, axes = plt.subplots(figsize=(len(y_pos) * 2, 5))
    rect1 = []
    rect2 = []
    rect3 = []
    rect4 = []
    rect5 = []
    rect5 = []
    rect7 = []
    rect8 = []
    rect9 = []
    rect10 = []
    width = 0.09
    rect1 = plt.bar(y_pos, metric_df['mean_absolute_error_train_rmse'], width, color=colors[0])
    rect2 = plt.bar(y_pos + width, metric_df['mean_squared_error_train_rmse'], width, color=colors[1])
    rect3 = plt.bar(y_pos + 2 * width, metric_df['root_mean_squared_error_train_rmse'], width, color=colors[2])
    rect4 = plt.bar(y_pos + 3 * width, metric_df['median_absolute_error_train_rmse'], width, color=colors[3])
    rect5 = plt.bar(y_pos + 4 * width, metric_df['r2_train_rmse'], width, color=colors[4])
    rect6 = plt.bar(y_pos + 5 * width, metric_df['mean_absolute_error_test_rmse'], width, color=colors[5])
    rect7 = plt.bar(y_pos + 6 * width, metric_df['mean_squared_error_test_rmse'], width, color=colors[6])
    rect8 = plt.bar(y_pos + 7 * width, metric_df['root_mean_squared_error_test_rmse'], width, color=colors[7])
    rect9 = plt.bar(y_pos + 8 * width, metric_df['median_absolute_error_test_rmse'], width, color=colors[8])
    rect10 = plt.bar(y_pos + 9 * width, metric_df['r2_test_rmse'], width, color=colors[9])

    plt.xticks(y_pos + 1.5 * width, metric_df['algo'], rotation=20, ha='right')
    plt.ylabel('RMSE')
    plt.xlabel(benchmark_title)
    # lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    lgd = plt.legend((rect1, rect2, rect3, rect4, rect5,rect6,rect7,rect8,rect9,rect10),
                     ("mean_absolute_error train MedAE","mean_squared_error train MedAE","root_mean_squared_error train MedAE","median_absolute_error train MedAE","r2 train MedAE"
                      , "mean_absolute_error test MedAE", "mean_squared_error test MedAE", "root_mean_squared_error test MedAE", "median_absolute_error test MedAE", "r2 test MedAE"),
                     bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    strFile = drive_location + benchmark_title + "_AllMetrix_rmse" + plot_file_ext
    if os.path.isfile(strFile):
        os.remove(strFile)
    # fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(strFile, bbox_inches='tight')

## comparision of all metric of pytorch not rmse

In [18]:
def capture_metric_in_csv_all_pytorch1(benchmark_title, algo_list):
  # First capture data in csv file for all algo
  metric_list = []
  for algo in algo_list:
    mean_absolute_error = load_learned_models(algo+"mean_absolute_error", "autopytorch")
    mean_squared_error = load_learned_models(algo+"mean_squared_error", "autopytorch")
    median_absolute_error = load_learned_models(algo + "median_absolute_error", "autopytorch")
    r2 = load_learned_models(algo + "r2", "autopytorch")
    metric_dict = {}
    metric_dict['algo'] = algo
    # RMSE
    metric_dict['mean_absolute_error_train_rmse'] = mean_absolute_error['train_rmse']
    metric_dict['mean_squared_error_train_rmse'] = mean_squared_error['train_rmse']
    metric_dict['median_absolute_error_train_rmse'] = median_absolute_error['train_rmse']
    metric_dict['r2_train_rmse'] = r2['train_rmse']

    metric_dict['mean_absolute_error_test_rmse'] = mean_absolute_error['test_rmse']
    metric_dict['mean_squared_error_test_rmse'] = mean_squared_error['test_rmse']
    metric_dict['median_absolute_error_test_rmse'] = median_absolute_error['test_rmse']
    metric_dict['r2_test_rmse'] = r2['test_rmse']
    # MedAE
    metric_dict['mean_absolute_error_train_medae'] = mean_absolute_error['train_medae']
    metric_dict['mean_squared_error_train_medae'] = mean_squared_error['train_medae']
    metric_dict['median_absolute_error_train_medae'] = median_absolute_error['train_medae']
    metric_dict['r2_train_medae'] = r2['train_medae']

    metric_dict['mean_absolute_error_test_medae'] = mean_absolute_error['test_medae']
    metric_dict['mean_squared_error_test_medae'] = mean_squared_error['test_medae']
    metric_dict['median_absolute_error_test_medae'] = median_absolute_error['test_medae']
    metric_dict['r2_test_medae'] = r2['test_medae']

    # MedAPE
    metric_dict['mean_absolute_error_train_medape'] = mean_absolute_error['train_medape']
    metric_dict['mean_squared_error_train_medape'] = mean_squared_error['train_medape']
    metric_dict['median_absolute_error_train_medape'] = median_absolute_error['train_medape']
    metric_dict['r2_train_medape'] = r2['train_medape']

    metric_dict['mean_absolute_error_test_medape'] = mean_absolute_error['test_medape']
    metric_dict['mean_squared_error_test_medape'] = mean_squared_error['test_medape']
    metric_dict['median_absolute_error_test_medape'] = median_absolute_error['test_medape']
    metric_dict['r2_test_medape'] = r2['test_medape']

    print(metric_dict)
    metric_list.append(metric_dict)
  metric_df = pd.DataFrame.from_dict(metric_list)
  metric_df.to_csv(drive_location+benchmark_title+"allAlgo"+"_metric.csv", line_terminator='\n', index=False)


In [19]:
def plot_all_pytorch1(benchmark_title, algo_list):
    capture_metric_in_csv_all_pytorch1(benchmark_title, algo_list)

    metric_df = pd.read_csv(drive_location + benchmark_title + "allAlgo" + "_metric.csv")
    y_pos = np.arange(metric_df.shape[0])
    print(y_pos)

    colors = ['r','k','b','y','g','r','k','b','y','g']
    fig, axes = plt.subplots(figsize=(len(y_pos) * 2, 4))
    rect1 = []
    rect2 = []
    rect3 = []
    rect4 = []
    rect5 = []
    rect5 = []
    rect7 = []
    rect8 = []
    width = 0.09
    rect1 = plt.bar(y_pos            , metric_df['mean_absolute_error_train_medae'], width, color=colors[0])
    rect2 = plt.bar(y_pos +    width, metric_df['mean_squared_error_train_medae'], width, color=colors[1])
    rect3 = plt.bar(y_pos + 2 * width, metric_df['median_absolute_error_train_medae'], width, color=colors[3])
    rect4 = plt.bar(y_pos + 3 * width, metric_df['r2_train_medae'], width, color=colors[4])
    rect5 = plt.bar(y_pos + 4 * width, metric_df['mean_absolute_error_test_medae'], width, color=colors[5])
    rect6 = plt.bar(y_pos + 5 * width, metric_df['mean_squared_error_test_medae'], width, color=colors[6])
    rect7 = plt.bar(y_pos + 6 * width, metric_df['median_absolute_error_test_medae'], width, color=colors[8])
    rect8 = plt.bar(y_pos + 7 * width, metric_df['r2_test_medae'], width, color=colors[9])

    plt.xticks(y_pos + 1 * width, metric_df['algo'], rotation=20, ha='right')
    plt.ylabel('MedAE')
    plt.xlabel(benchmark_title)
    # lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    lgd = plt.legend((rect1, rect2, rect3, rect4, rect5, rect6, rect7, rect8,),
                     ("mean_absolute_error train MedAE", "mean_squared_error train MedAE",
                       "median_absolute_error train MedAE", "r2 train MedAE"
                      , "mean_absolute_error test MedAE", "mean_squared_error test MedAE",
                       "median_absolute_error test MedAE", "r2 test MedAE"),
                     bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    strFile = drive_location + benchmark_title + "_AllMetrix_MedAE" + plot_file_ext
    if os.path.isfile(strFile):
        os.remove(strFile)
    # fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(strFile, bbox_inches='tight')

    # Plot Train and Test MedAPE for nnmodel (our model) and autokeras model
    colors = ['r','k','b','y','g','r','k','b','y','g']
    fig, axes = plt.subplots(figsize=(len(y_pos) * 2, 4))
    rect1 = []
    rect2 = []
    rect3 = []
    rect4 = []
    rect5 = []
    rect5 = []
    rect7 = []
    rect8 = []
    width = 0.09
    rect1 = plt.bar(y_pos, metric_df['mean_absolute_error_train_medape'], width, color=colors[0])
    rect2 = plt.bar(y_pos + width, metric_df['mean_squared_error_train_medape'], width, color=colors[1])
    rect3 = plt.bar(y_pos + 2 * width, metric_df['median_absolute_error_train_medape'], width, color=colors[3])
    rect4 = plt.bar(y_pos + 3 * width, metric_df['r2_train_medape'], width, color=colors[4])
    rect5 = plt.bar(y_pos + 4 * width, metric_df['mean_absolute_error_test_medape'], width, color=colors[5])
    rect6 = plt.bar(y_pos + 5 * width, metric_df['mean_squared_error_test_medape'], width, color=colors[6])
    rect7 = plt.bar(y_pos + 6 * width, metric_df['median_absolute_error_test_medape'], width, color=colors[8])
    rect8 = plt.bar(y_pos + 7 * width, metric_df['r2_test_medape'], width, color=colors[9])

    plt.xticks(y_pos + 1.5 * width, metric_df['algo'], rotation=20, ha='right')
    plt.ylabel('MedAPE')
    plt.xlabel(benchmark_title)
    # lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    lgd = plt.legend((rect1, rect2, rect3, rect4, rect5, rect6, rect7, rect8),
                     ("mean_absolute_error train MedAE", "mean_squared_error train MedAE",
                       "median_absolute_error train MedAE", "r2 train MedAE"
                      , "mean_absolute_error test MedAE", "mean_squared_error test MedAE",
                       "median_absolute_error test MedAE", "r2 test MedAE"),
                     bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    strFile = drive_location + benchmark_title + "_AllMetrix_medape" + plot_file_ext
    if os.path.isfile(strFile):
        os.remove(strFile)
    # fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(strFile, bbox_inches='tight')

    # drop RMSE values for SPEC2006 Int Benchmarks due to very high value
    if (benchmark_title == 'SPEC_Int'):
        indexNames = metric_df[
            metric_df['algo'].isin(['401.bzip2', '456.hmmer', '458.sjeng', '462.libquantum', '473.astar'])].index
        # print(indexNames)
        metric_df = metric_df.drop(indexNames)
        # print(metric_df)
    y_pos = np.arange(metric_df.shape[0])
    print(y_pos)

    # Plot Train and Test RMSE for nnmodel (our model) and autokeras model
    colors = ['r','k','b','y','g','r','k','b','y','g']
    fig, axes = plt.subplots(figsize=(len(y_pos) * 2, 5))
    rect1 = []
    rect2 = []
    rect3 = []
    rect4 = []
    rect5 = []
    rect5 = []
    rect7 = []
    rect8 = []
    width = 0.09
    rect1 = plt.bar(y_pos, metric_df['mean_absolute_error_train_rmse'], width, color=colors[0])
    rect2 = plt.bar(y_pos + width, metric_df['mean_squared_error_train_rmse'], width, color=colors[1])
    rect3 = plt.bar(y_pos + 2 * width, metric_df['median_absolute_error_train_rmse'], width, color=colors[3])
    rect4 = plt.bar(y_pos + 3 * width, metric_df['r2_train_rmse'], width, color=colors[4])
    rect5 = plt.bar(y_pos + 4 * width, metric_df['mean_absolute_error_test_rmse'], width, color=colors[5])
    rect6 = plt.bar(y_pos + 5 * width, metric_df['mean_squared_error_test_rmse'], width, color=colors[6])
    rect7 = plt.bar(y_pos + 6 * width, metric_df['median_absolute_error_test_rmse'], width, color=colors[8])
    rect8 = plt.bar(y_pos + 7 * width, metric_df['r2_test_rmse'], width, color=colors[9])

    plt.xticks(y_pos + 1.5 * width, metric_df['algo'], rotation=20, ha='right')
    plt.ylabel('RMSE')
    plt.xlabel(benchmark_title)
    # lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    lgd = plt.legend((rect1, rect2, rect3, rect4, rect5,rect6,rect7,rect8),
                     ("mean_absolute_error train MedAE","mean_squared_error train MedAE","median_absolute_error train MedAE","r2 train MedAE"
                      , "mean_absolute_error test MedAE", "mean_squared_error test MedAE", "median_absolute_error test MedAE", "r2 test MedAE"),
                     bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    strFile = drive_location + benchmark_title + "_AllMetrix_rmse" + plot_file_ext
    if os.path.isfile(strFile):
        os.remove(strFile)
    # fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(strFile, bbox_inches='tight')

## nn Vs autoKeras Vs autoPytorch

In [20]:
def capture_metric_in_csv_NN_Vs_AutoKeras_autoPytorch(benchmark_title, algo_list,optimizeMetric='r2'):
  # First capture data in csv file for all algo
  metric_list = []
  for algo in algo_list:
    nnmodel = load_learned_models(algo, "nnmodel")
    final_nnmodel_key = list(nnmodel.keys())[-1]
    autokerasmodel = load_learned_models(algo, "autokeras")
    autopytorchmodel = load_learned_models(algo+optimizeMetric, "autopytorch")
    metric_dict = {}
    metric_dict['algo'] = algo
    # RMSE
    metric_dict['nnmodel_train_rmse'] = nnmodel[final_nnmodel_key]['train_rmse']
    metric_dict['autokerasmodel_train_rmse'] = autokerasmodel['train_rmse']
    metric_dict['autopytorchmodel_train_rmse'] = autopytorchmodel['train_rmse']
    metric_dict['nnmodel_test_rmse'] = nnmodel[final_nnmodel_key]['test_rmse']
    metric_dict['autokerasmodel_test_rmse'] = autokerasmodel['test_rmse']
    metric_dict['autopytorchmodel_test_rmse'] = autopytorchmodel['test_rmse']
    # MedAE
    metric_dict['nnmodel_train_medae'] = nnmodel[final_nnmodel_key]['train_medae']
    metric_dict['autokerasmodel_train_medae'] = autokerasmodel['train_medae']
    metric_dict['autopytorchmodel_train_medae'] = autopytorchmodel['train_medae']
    metric_dict['nnmodel_test_medae'] = nnmodel[final_nnmodel_key]['test_medae']
    metric_dict['autokerasmodel_test_medae'] = autokerasmodel['test_medae']
    metric_dict['autopytorchmodel_test_medae'] = autopytorchmodel['test_medae']
    # MedAPE
    metric_dict['nnmodel_train_medape'] = nnmodel[final_nnmodel_key]['train_medape']
    metric_dict['autokerasmodel_train_medape'] = autokerasmodel['train_medape']
    metric_dict['autopytorchmodel_train_medape'] = autopytorchmodel['train_medape']
    metric_dict['nnmodel_test_medape'] = nnmodel[final_nnmodel_key]['test_medape']
    metric_dict['autokerasmodel_test_medape'] = autokerasmodel['test_medape']
    metric_dict['autopytorchmodel_test_medape'] = autopytorchmodel['test_medape']

    print(metric_dict)
    metric_list.append(metric_dict)
  metric_df = pd.DataFrame.from_dict(metric_list)
  metric_df.to_csv(drive_location+benchmark_title+"_metric.csv", line_terminator='\n', index=False)

In [21]:
def plot_nnmodel_vs_autokeras_autoPytorch(benchmark_title, algo_list,optimizeMetri='r2'):
    capture_metric_in_csv_NN_Vs_AutoKeras_autoPytorch(benchmark_title, algo_list,optimizeMetri)

    metric_df = pd.read_csv(drive_location + benchmark_title + "_metric.csv")
    y_pos = np.arange(metric_df.shape[0])
    print(y_pos)

    # Plot Train and Test MedAE for nnmodel (our model) and autokeras model
    colors = ['r', 'k', 'b', 'r', 'k', 'b']
    fig, axes = plt.subplots(figsize=(len(y_pos) * 1.5, 6))
    rect1 = []
    rect2 = []
    rect3 = []
    rect4 = []
    rect5 = []
    rect6 = []
    width = 0.1
    rect1 = plt.bar(y_pos, metric_df['nnmodel_train_medae'], width, color=colors[0])
    rect2 = plt.bar(y_pos + width, metric_df['autokerasmodel_train_medae'], width, color=colors[1])
    rect3 = plt.bar(y_pos + 2 * width, metric_df['autopytorchmodel_train_medae'], width, color=colors[2])
    rect4 = plt.bar(y_pos + 3 * width, metric_df['nnmodel_test_medae'], width, color=colors[3])
    rect5 = plt.bar(y_pos + 4 * width, metric_df['autokerasmodel_test_medae'], width, color=colors[4])
    rect6 = plt.bar(y_pos + 5 * width, metric_df['autopytorchmodel_test_medae'], width, color=colors[5])

    plt.xticks(y_pos + 1.5 * width, metric_df['algo'], rotation=20, ha='right')
    plt.ylabel('MedAE')
    plt.xlabel(benchmark_title)
    # lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    lgd = plt.legend((rect1, rect2, rect3, rect4, rect5, rect6), (
    "Our Model Train MedAE", "Autokeras Model Train MedAE", "AutoPytorch Model Train MedAE", "Our Model Test MedAE",
    "Autokeras Test MedAE", "AutoPytorch Test MedAE"), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    strFile = drive_location + benchmark_title + "_nnmodel_vs_autokerasmodel_vs_autoPytorch_medae" + plot_file_ext
    if os.path.isfile(strFile):
        os.remove(strFile)
    # fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(strFile, bbox_inches='tight')

    # Plot Train and Test MedAPE for nnmodel (our model) and autokeras model
    colors = ['r', 'k', 'b','r', 'k', 'b']
    fig, axes = plt.subplots(figsize=(len(y_pos) * 1.5, 6))
    rect1 = []
    rect2 = []
    rect3 = []
    rect4 = []
    rect5 = []
    rect6 = []
    width = 0.1
    rect1 = plt.bar(y_pos, metric_df['nnmodel_train_medape'], width, color=colors[0])
    rect2 = plt.bar(y_pos + width, metric_df['autokerasmodel_train_medape'], width, color=colors[1])
    rect3 = plt.bar(y_pos + 2 * width, metric_df['autopytorchmodel_train_medape'], width, color=colors[2])
    rect4 = plt.bar(y_pos + 3 * width, metric_df['nnmodel_test_medape'], width, color=colors[3])
    rect5 = plt.bar(y_pos + 4 * width, metric_df['autokerasmodel_test_medape'], width, color=colors[4])
    rect6 = plt.bar(y_pos + 5 * width, metric_df['autopytorchmodel_test_medape'], width, color=colors[5])

    plt.xticks(y_pos + 1.5 * width, metric_df['algo'], rotation=20, ha='right')
    plt.ylabel('MedAPE')
    plt.xlabel(benchmark_title)
    # lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    lgd = plt.legend((rect1, rect2, rect3, rect4, rect5, rect6), (
    "Our Model Train MedAPE", "Autokeras Model Train MedAPE", "AutoPytorch Model Train MedAPE"
    , "Our Model Test MedAPE", "Autokeras Test MedAPE", "AutoPytorch Test MedAPE"),
                     bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    strFile = drive_location + benchmark_title + "_nnmodel_vs_autokerasmodel_vs_autoPytorch_medape" + plot_file_ext
    if os.path.isfile(strFile):
        os.remove(strFile)
    # fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(strFile, bbox_inches='tight')

    # drop RMSE values for SPEC2006 Int Benchmarks due to very high value
    if (benchmark_title == 'SPEC_Int'):
        indexNames = metric_df[
            metric_df['algo'].isin(['401.bzip2', '456.hmmer', '458.sjeng', '462.libquantum', '473.astar'])].index
        # print(indexNames)
        metric_df = metric_df.drop(indexNames)
        # print(metric_df)
    y_pos = np.arange(metric_df.shape[0])
    print(y_pos)

    # Plot Train and Test RMSE for nnmodel (our model) and autokeras model
    colors = ['r', 'k', 'b','r', 'k', 'b']
    fig, axes = plt.subplots(figsize=(len(y_pos) * 1.5, 6))
    rect1 = []
    rect2 = []
    rect3 = []
    rect4 = []
    rect5 = []
    rect6 = []
    width = 0.1
    rect1 = plt.bar(y_pos, metric_df['nnmodel_train_rmse'], width, color=colors[0])
    rect2 = plt.bar(y_pos + width, metric_df['autokerasmodel_train_rmse'], width, color=colors[1])
    rect3 = plt.bar(y_pos + 2 * width, metric_df['autopytorchmodel_train_rmse'], width, color=colors[2])
    rect4 = plt.bar(y_pos + 3 * width, metric_df['nnmodel_test_rmse'], width, color=colors[3])
    rect5 = plt.bar(y_pos + 4 * width, metric_df['autokerasmodel_test_rmse'], width, color=colors[4])
    rect6 = plt.bar(y_pos + 5 * width, metric_df['autopytorchmodel_test_rmse'], width, color=colors[5])

    plt.xticks(y_pos + 1.5 * width, metric_df['algo'], rotation=20, ha='right')
    plt.ylabel('RMSE')
    plt.xlabel(benchmark_title)
    # lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    lgd = plt.legend((rect1, rect2, rect3, rect4, rect5, rect6), (
    "Our Model Train RMSE", "Autokeras Model Train RMSE","AutoPytorch Model Train RMSE", "Our Model Test RMSE", "Autokeras Test RMSE",
    "AutoPytorch Test RMSE"), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    strFile = drive_location + benchmark_title + "_nnmodel_vs_autokerasmodel_vs_autoPytorch_rmse" + plot_file_ext
    if os.path.isfile(strFile):
        os.remove(strFile)
    # fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(strFile, bbox_inches='tight')

## nn Vs autoKeras Vs autoPytorch VS MLJar

In [22]:
def capture_metric_in_csv_NN_Vs_AutoKeras_autoPytorch_automljar(benchmark_title, algo_list,optimizeMetric='r2'):
  # First capture data in csv file for all algo
  metric_list = []
  for algo in algo_list:
    nnmodel = load_learned_models(algo, "nnmodel")
    final_nnmodel_key = list(nnmodel.keys())[-1]
    autokerasmodel = load_learned_models(algo, "autokeras")
    autopytorchmodel = load_learned_models(algo+optimizeMetric, "autopytorch")
    automljarmodel = load_learned_models(algo, "automljar")
    metric_dict = {}
    metric_dict['algo'] = algo
    # RMSE
    metric_dict['nnmodel_train_rmse'] = nnmodel[final_nnmodel_key]['train_rmse']
    metric_dict['autokerasmodel_train_rmse'] = autokerasmodel['train_rmse']
    metric_dict['autopytorchmodel_train_rmse'] = autopytorchmodel['train_rmse']
    metric_dict['automljarmodel_train_rmse'] = automljarmodel['train_rmse']
    metric_dict['nnmodel_test_rmse'] = nnmodel[final_nnmodel_key]['test_rmse']
    metric_dict['autokerasmodel_test_rmse'] = autokerasmodel['test_rmse']
    metric_dict['autopytorchmodel_test_rmse'] = autopytorchmodel['test_rmse']
    metric_dict['automljarmodel_test_rmse'] = automljarmodel['test_rmse']
    # MedAE
    metric_dict['nnmodel_train_medae'] = nnmodel[final_nnmodel_key]['train_medae']
    metric_dict['autokerasmodel_train_medae'] = autokerasmodel['train_medae']
    metric_dict['autopytorchmodel_train_medae'] = autopytorchmodel['train_medae']
    metric_dict['automljarmodel_train_medae'] = automljarmodel['train_medae']
    metric_dict['nnmodel_test_medae'] = nnmodel[final_nnmodel_key]['test_medae']
    metric_dict['autokerasmodel_test_medae'] = autokerasmodel['test_medae']
    metric_dict['autopytorchmodel_test_medae'] = autopytorchmodel['test_medae']
    metric_dict['automljarmodel_test_medae'] = automljarmodel['test_medae']
    # MedAPE
    metric_dict['nnmodel_train_medape'] = nnmodel[final_nnmodel_key]['train_medape']
    metric_dict['autokerasmodel_train_medape'] = autokerasmodel['train_medape']
    metric_dict['autopytorchmodel_train_medape'] = autopytorchmodel['train_medape']
    metric_dict['automljarmodel_train_medape'] = automljarmodel['train_medape']
    metric_dict['nnmodel_test_medape'] = nnmodel[final_nnmodel_key]['test_medape']
    metric_dict['autokerasmodel_test_medape'] = autokerasmodel['test_medape']
    metric_dict['autopytorchmodel_test_medape'] = autopytorchmodel['test_medape']
    metric_dict['automljarmodel_test_medape'] = automljarmodel['test_medape']

    print(metric_dict)
    metric_list.append(metric_dict)
  metric_df = pd.DataFrame.from_dict(metric_list)
  metric_df.to_csv(drive_location+benchmark_title+"_metric.csv", line_terminator='\n', index=False)

In [23]:
def plot_nnmodel_vs_autokeras_autoPytorch_mljar(benchmark_title, algo_list,optimizeMetri='r2'):
    capture_metric_in_csv_NN_Vs_AutoKeras_autoPytorch_automljar(benchmark_title, algo_list,optimizeMetri)

    metric_df = pd.read_csv(drive_location + benchmark_title + "_metric.csv")
    y_pos = np.arange(metric_df.shape[0])
    print(y_pos)

    # Plot Train and Test MedAE for nnmodel (our model) and autokeras model
    # colors = ['r', 'k', 'b', 'y','r', 'k', 'b', 'y',]
    # fig, axes = plt.subplots(figsize=(len(y_pos) * 1.5, 6))
    # rect1 = []
    # rect2 = []
    # rect3 = []
    # rect4 = []
    # rect5 = []
    # rect6 = []
    # rect7 = []
    # rect8 = []
    # width = 0.1
    # rect1 = plt.bar(y_pos, metric_df['nnmodel_train_medae'], width, color=colors[0])
    # rect2 = plt.bar(y_pos + width, metric_df['autokerasmodel_train_medae'], width, color=colors[1])
    # rect3 = plt.bar(y_pos + 2 * width, metric_df['autopytorchmodel_train_medae'], width, color=colors[2])
    # rect4 = plt.bar(y_pos + 3 * width, metric_df['automljarmodel_train_medae'], width, color=colors[3])
    # rect5 = plt.bar(y_pos + 4 * width, metric_df['nnmodel_test_medae'], width, color=colors[4])
    # rect6 = plt.bar(y_pos + 5 * width, metric_df['autokerasmodel_test_medae'], width, color=colors[5])
    # rect7 = plt.bar(y_pos + 6 * width, metric_df['autopytorchmodel_test_medae'], width, color=colors[6])
    # rect8 = plt.bar(y_pos + 7 * width, metric_df['automljarmodel_test_medae'], width, color=colors[7])


    # plt.xticks(y_pos + 1.5 * width, metric_df['algo'], rotation=20, ha='right')
    # plt.ylabel('MedAE')
    # plt.xlabel(benchmark_title)
    # # lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    # lgd = plt.legend((rect1, rect2, rect3, rect4, rect5, rect6,rect7,rect8), (
    # "Our Model Train MedAE", "Autokeras Model Train MedAE", "AutoPytorch Model Train MedAE","AutoMlJar Model Train MedAE",
    # "Our Model Test MedAE","Autokeras Test MedAE", "AutoPytorch Test MedAE", "AutoMlJar Test MedAE"), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    # plt.show()
    # strFile = drive_location + benchmark_title + "_nnmodel_vs_autokerasmodel_vs_autoPytorch_vs_automlJar_medae" + plot_file_ext
    # if os.path.isfile(strFile):
    #     os.remove(strFile)
    # # fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    # fig.savefig(strFile, bbox_inches='tight')

    algosList = ["train_medae",'test_medae']
    algo_names = ['nnmodel','autokerasmodel','autopytorchmodel','automljarmodel']
    color = ['r', 'k', 'b', 'y']
    algo_names_list= (   "Our Model Train MedAE", "Autokeras Model Train MedAE", "AutoPytorch Model Train MedAE","AutoMlJar Model Train MedAE",    "Our Model Test MedAE","Autokeras Test MedAE", "AutoPytorch Test MedAE", "AutoMlJar Test MedAE")
    label ='MedAE'
    name="_nnmodel_vs_autokerasmodel_vs_autoPytorch_vs_automlJar_medae"
    make_graph(benchmark_title, algo_list,algosList,algo_names,metric_df,color,algo_names_list,label,name)

    # # Plot Train and Test MedAPE for nnmodel (our model) and autokeras model
    # colors = ['r', 'k', 'b', 'y','r', 'k', 'b', 'y',]
    # fig, axes = plt.subplots(figsize=(len(y_pos) * 1.5, 6))
    # rect1 = []
    # rect2 = []
    # rect3 = []
    # rect4 = []
    # rect5 = []
    # rect6 = []
    # rect7 = []
    # rect8 = []

    # width = 0.1
    # rect1 = plt.bar(y_pos, metric_df['nnmodel_train_medape'], width, color=colors[0])
    # rect2 = plt.bar(y_pos + width, metric_df['autokerasmodel_train_medape'], width, color=colors[1])
    # rect3 = plt.bar(y_pos + 2 * width, metric_df['autopytorchmodel_train_medape'], width, color=colors[2])
    # rect4 = plt.bar(y_pos + 3 * width, metric_df['automljarmodel_train_medape'], width, color=colors[3])
    # rect5 = plt.bar(y_pos + 4 * width, metric_df['nnmodel_test_medape'], width, color=colors[4])
    # rect6 = plt.bar(y_pos + 5 * width, metric_df['autokerasmodel_test_medape'], width, color=colors[5])
    # rect7 = plt.bar(y_pos + 6 * width, metric_df['autopytorchmodel_test_medape'], width, color=colors[6])
    # rect8 = plt.bar(y_pos + 7 * width, metric_df['automljarmodel_test_medape'], width, color=colors[7])

    # plt.xticks(y_pos + 1.5 * width, metric_df['algo'], rotation=20, ha='right')
    # plt.ylabel('MedAPE')
    # plt.xlabel(benchmark_title)
    # # lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    # lgd = plt.legend((rect1, rect2, rect3, rect4, rect5, rect6,rect7,rect8), (
    # "Our Model Train MedAPE", "Autokeras Model Train MedAPE", "AutoPytorch Model Train MedAPE","AutoMlJar Model Train MedAPE"
    # , "Our Model Test MedAPE", "Autokeras Test MedAPE", "AutoPytorch Test MedAPE", "AutoMlJar Test MedAPE"),
    #                  bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    # plt.show()
    # strFile = drive_location + benchmark_title + "_nnmodel_vs_autokerasmodel_vs_autoPytorch_vs_automlJar_medape" + plot_file_ext
    # if os.path.isfile(strFile):
    #     os.remove(strFile)
    # # fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    # fig.savefig(strFile, bbox_inches='tight')

    algosList = ["train_medape",'test_medape']
    algo_names = ['nnmodel','autokerasmodel','autopytorchmodel','automljarmodel']
    color = ['r', 'k', 'b', 'y']
    algo_names_list= ( "Our Model Train MedAPE", "Autokeras Model Train MedAPE", "AutoPytorch Model Train MedAPE","AutoMlJar Model Train MedAPE" , "Our Model Test MedAPE", "Autokeras Test MedAPE", "AutoPytorch Test MedAPE", "AutoMlJar Test MedAPE")
    label ='MedAPE'
    name="_nnmodel_vs_autokerasmodel_vs_autoPytorch_vs_automlJar_medape"
    make_graph(benchmark_title, algo_list,algosList,algo_names,metric_df,color,algo_names_list,label,name)


    # # drop RMSE values for SPEC2006 Int Benchmarks due to very high value
    # if (benchmark_title == 'SPEC_Int'):
    #     indexNames = metric_df[
    #         metric_df['algo'].isin(['401.bzip2', '456.hmmer', '458.sjeng', '462.libquantum', '473.astar'])].index
    #     # print(indexNames)
    #     metric_df = metric_df.drop(indexNames)
    #     # print(metric_df)
    # y_pos = np.arange(metric_df.shape[0])
    # print(y_pos)

    # # Plot Train and Test RMSE for nnmodel (our model) and autokeras model
    # colors = ['r', 'k', 'b', 'y','r', 'k', 'b', 'y',]
    # fig, axes = plt.subplots(figsize=(len(y_pos) * 1.5, 6))
    # rect1 = []
    # rect2 = []
    # rect3 = []
    # rect4 = []
    # rect5 = []
    # rect6 = []
    # rect7 = []
    # rect8 = []
    # width = 0.1
    # rect1 = plt.bar(y_pos, metric_df['nnmodel_train_rmse'], width, color=colors[0])
    # rect2 = plt.bar(y_pos + width, metric_df['autokerasmodel_train_rmse'], width, color=colors[1])
    # rect3 = plt.bar(y_pos + 2 * width, metric_df['autopytorchmodel_train_rmse'], width, color=colors[2])
    # rect4 = plt.bar(y_pos + 3 * width, metric_df['automljarmodel_train_rmse'], width, color=colors[3])
    # rect5 = plt.bar(y_pos + 4 * width, metric_df['nnmodel_test_rmse'], width, color=colors[4])
    # rect6 = plt.bar(y_pos + 5 * width, metric_df['autokerasmodel_test_rmse'], width, color=colors[5])
    # rect7 = plt.bar(y_pos + 6 * width, metric_df['autopytorchmodel_test_rmse'], width, color=colors[6])
    # rect8 = plt.bar(y_pos + 7 * width, metric_df['automljarmodel_test_rmse'], width, color=colors[7])

    # plt.xticks(y_pos + 1.5 * width, metric_df['algo'], rotation=20, ha='right')
    # plt.ylabel('RMSE')
    # plt.xlabel(benchmark_title)
    # # lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    # lgd = plt.legend((rect1, rect2, rect3, rect4, rect5, rect6,rect7,rect8), (
    # "Our Model Train RMSE", "Autokeras Model Train RMSE","AutoPytorch Model Train RMSE","AutoMlJar Model Train RMSE", 
    # "Our Model Test RMSE", "Autokeras Test RMSE","AutoPytorch Test RMSE","AutoMlJar Test RMSE"), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    # plt.show()
    # strFile = drive_location + benchmark_title + "_nnmodel_vs_autokerasmodel_vs_autoPytorch_vs_automlJar_rmse" + plot_file_ext
    # if os.path.isfile(strFile):
    #     os.remove(strFile)
    # # fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    # fig.savefig(strFile, bbox_inches='tight')

    algosList = ["train_rmse",'test_rmse']
    algo_names = ['nnmodel','autokerasmodel','autopytorchmodel','automljarmodel']
    color = ['r', 'k', 'b', 'y']
    algo_names_list= ("Our Model Train RMSE", "Autokeras Model Train RMSE","AutoPytorch Model Train RMSE","AutoMlJar Model Train RMSE", "Our Model Test RMSE", "Autokeras Test RMSE","AutoPytorch Test RMSE","AutoMlJar Test RMSE")
    label ='RMSE'
    name="_nnmodel_vs_autokerasmodel_vs_autoPytorch_vs_automlJar_rmse"
    make_graph(benchmark_title, algo_list,algosList,algo_names,metric_df,color,algo_names_list,label,name)

### ploat graph

In [24]:
def MakeCSV(benchmark_title, algo_list,optimizeMetric='r2'):
  # First capture data in csv file for all algo
  metric_list = []
  for algo in algo_list:
    nnmodel = load_learned_models(algo, "nnmodel")
    final_nnmodel_key = list(nnmodel.keys())[-1]
    autokerasmodel = load_learned_models(algo, "autokeras")
    autopytorchmodel = load_learned_models(algo+optimizeMetric, "autopytorch")
    automljarmodel = load_learned_models(algo, "automljar")
    metric_dict = {}
    metric_dict['algo'] = algo
    # RMSE
    metric_dict['nnmodel_train_rmse'] = nnmodel[final_nnmodel_key]['train_rmse']
    metric_dict['autokerasmodel_train_rmse'] = autokerasmodel['train_rmse']
    metric_dict['autopytorchmodel_train_rmse'] = autopytorchmodel['train_rmse']
    metric_dict['automljarmodel_train_rmse'] = automljarmodel['train_rmse']
    metric_dict['nnmodel_test_rmse'] = nnmodel[final_nnmodel_key]['test_rmse']
    metric_dict['autokerasmodel_test_rmse'] = autokerasmodel['test_rmse']
    metric_dict['autopytorchmodel_test_rmse'] = autopytorchmodel['test_rmse']
    metric_dict['automljarmodel_test_rmse'] = automljarmodel['test_rmse']
    # MedAE
    metric_dict['nnmodel_train_medae'] = nnmodel[final_nnmodel_key]['train_medae']
    metric_dict['autokerasmodel_train_medae'] = autokerasmodel['train_medae']
    metric_dict['autopytorchmodel_train_medae'] = autopytorchmodel['train_medae']
    metric_dict['automljarmodel_train_medae'] = automljarmodel['train_medae']
    metric_dict['nnmodel_test_medae'] = nnmodel[final_nnmodel_key]['test_medae']
    metric_dict['autokerasmodel_test_medae'] = autokerasmodel['test_medae']
    metric_dict['autopytorchmodel_test_medae'] = autopytorchmodel['test_medae']
    metric_dict['automljarmodel_test_medae'] = automljarmodel['test_medae']
    # MedAPE
    metric_dict['nnmodel_train_medape'] = nnmodel[final_nnmodel_key]['train_medape']
    metric_dict['autokerasmodel_train_medape'] = autokerasmodel['train_medape']
    metric_dict['autopytorchmodel_train_medape'] = autopytorchmodel['train_medape']
    metric_dict['automljarmodel_train_medape'] = automljarmodel['train_medape']
    metric_dict['nnmodel_test_medape'] = nnmodel[final_nnmodel_key]['test_medape']
    metric_dict['autokerasmodel_test_medape'] = autokerasmodel['test_medape']
    metric_dict['autopytorchmodel_test_medape'] = autopytorchmodel['test_medape']
    metric_dict['automljarmodel_test_medape'] = automljarmodel['test_medape']

    print(metric_dict)
    metric_list.append(metric_dict)
  metric_df = pd.DataFrame.from_dict(metric_list)
  metric_df.to_csv(drive_location+benchmark_title+"_metric.csv", line_terminator='\n', index=False)

In [25]:
def make_graph(benchmark_title, algo_list,algosList,algo_names,metric_df,colors,algo_names_list,label,name):
  # drop RMSE values for SPEC2006 Int Benchmarks due to very high value
    if (benchmark_title == 'SPEC_Int'):
        indexNames = metric_df[
            metric_df['algo'].isin(['401.bzip2', '456.hmmer', '458.sjeng', '462.libquantum', '473.astar'])].index
        # print(indexNames)
        metric_df = metric_df.drop(indexNames)
        # print(metric_df)
    y_pos = np.arange(metric_df.shape[0])
    print(y_pos)

    # Plot Train and Test RMSE for nnmodel (our model) and autokeras model
    fig, axes = plt.subplots(figsize=(len(y_pos) * 1.5, 6))
    width = 0.1
    rect = []

    i=0
    for algo in algosList:
      j=0
      for name in algo_names:
        rect.append(plt.bar(y_pos + i * width, metric_df[name+"_"+algo], width, color=colors[j]))
        i=i+1
        j=j+1
  
    plt.xticks(y_pos + 1.5 * width, metric_df['algo'], rotation=20, ha='right')
    plt.ylabel(label)
    plt.xlabel(benchmark_title)
    # lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    lgd = plt.legend((i for i in rect), algo_names_list , bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    strFile = drive_location + benchmark_title + name + plot_file_ext
    if os.path.isfile(strFile):
        os.remove(strFile)
    # fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(strFile, bbox_inches='tight')

## ploat type 2

In [26]:
def capture_iteration_metric_in_csv(benchmark_title, algo_list):
  # First capture data in csv file for all algo
  metric_list_train_rmse = []
  metric_list_train_medae = []
  metric_list_train_medape = []
  metric_list_test_rmse = []
  metric_list_test_medae = []
  metric_list_test_medape = []
  for algo in algo_list:
    nnmodel = load_learned_models(algo, "nnmodel")
    nnmodel_keys = list(nnmodel.keys())
    print(nnmodel_keys)
    metric_dict_train_rmse = {}
    metric_dict_train_medae = {}
    metric_dict_train_medape = {}
    metric_dict_test_rmse = {}
    metric_dict_test_medae = {}
    metric_dict_test_medape = {}

    metric_dict_train_rmse['algo'] = algo
    metric_dict_train_medae['algo'] = algo
    metric_dict_train_medape['algo'] = algo
    metric_dict_test_rmse['algo'] = algo
    metric_dict_test_medae['algo'] = algo
    metric_dict_test_medape['algo'] = algo
    
    for key in nnmodel_keys:
      # RMSE
      metric_dict_train_rmse[key] = nnmodel[key]['train_rmse']
      metric_dict_test_rmse[key] = nnmodel[key]['test_rmse']
      # MedAE
      metric_dict_train_medae[key] = nnmodel[key]['train_medae']
      metric_dict_test_medae[key] = nnmodel[key]['test_medae']
      # MedAPE
      metric_dict_train_medape[key] = nnmodel[key]['train_medape']
      metric_dict_test_medape[key] = nnmodel[key]['test_medape']

    print(metric_dict_train_rmse)
    metric_list_train_rmse.append(metric_dict_train_rmse)
    metric_df_train_rmse = pd.DataFrame.from_dict(metric_list_train_rmse)
    metric_df_train_rmse.to_csv(drive_location+benchmark_title+"_metric_train_rmse.csv", line_terminator='\n', index=False)

    metric_list_test_rmse.append(metric_dict_test_rmse)
    metric_df_test_rmse = pd.DataFrame.from_dict(metric_list_test_rmse)
    metric_df_test_rmse.to_csv(drive_location+benchmark_title+"_metric_test_rmse.csv", line_terminator='\n', index=False)

    metric_list_train_medae.append(metric_dict_train_medae)
    metric_df_train_medae = pd.DataFrame.from_dict(metric_list_train_medae)
    metric_df_train_medae.to_csv(drive_location+benchmark_title+"_metric_train_medae.csv", line_terminator='\n', index=False)

    metric_list_test_medae.append(metric_dict_test_medae)
    metric_df_test_medae = pd.DataFrame.from_dict(metric_list_test_medae)
    metric_df_test_medae.to_csv(drive_location+benchmark_title+"_metric_test_medae.csv", line_terminator='\n', index=False)

    metric_list_train_medape.append(metric_dict_train_medape)
    metric_df_train_medape = pd.DataFrame.from_dict(metric_list_train_medape)
    metric_df_train_medape.to_csv(drive_location+benchmark_title+"_metric_train_medape.csv", line_terminator='\n', index=False)

    metric_list_test_medape.append(metric_dict_test_medape)
    metric_df_test_medape = pd.DataFrame.from_dict(metric_list_test_medape)
    metric_df_test_medape.to_csv(drive_location+benchmark_title+"_metric_test_medape.csv", line_terminator='\n', index=False) 

In [27]:
def plot_iteration_metric(benchmark_title, algo_list):
  capture_iteration_metric_in_csv(benchmark_title, algo_list)

  metrics = ['train_rmse','test_rmse','train_medae','test_medae','train_medape','test_medape']

  for metric in metrics:
    metric_df = pd.read_csv(drive_location+benchmark_title+"_metric_"+metric+".csv")

    fig, axes = plt.subplots(1,1)
    for algo_idx, algo in enumerate(metric_df['algo']):
      print(metric_df.values[algo_idx][1:])
      x_pos = np.arange(len(metric_df.values[algo_idx][1:]))
      print(x_pos)
      plt.plot(x_pos, metric_df.values[algo_idx][1:])
    plt.xticks(x_pos)
    plt.ylabel(metric.upper())
    plt.xlabel('Iterations')
    plt.title('Iterations vs '+metric.upper())
    ldg = plt.legend(metric_df['algo'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    strFile = drive_location + benchmark_title + "_metric_"+metric+plot_file_ext
    if os.path.isfile(strFile):
      os.remove(strFile)
    #fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(strFile, bbox_inches='tight')

# import Packages

1. pandas
1. categrory_encoders
1. sklearn
1. pickle
1. gc
1. numpy


1. tensorflow
1. autokeras
1. keras-tuner

In [28]:
!pip install category_encoders
!pip install git+https://github.com/keras-team/keras-tuner.git
!pip install autokeras
!pip install autoPyTorch
!pip install mljar-supervised

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/keras-team/keras-tuner.git to /tmp/pip-req-build-fj4w5uzp
  Running command git clone -q https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-fj4w5uzp
  Created wheel for keras-tuner: filename=keras_tuner-1.1.3-py3-none-any.whl size=135701 sha256=39db75dc6d0121a66264406d6648e75e68e5418950ca5c095c9252063c91fd9e
  Stored in directory: /tmp/pip-ephem-wheel-cache-5dzbtwl3/wheels/e1/66/9f/f6c4d78f995ffaab8f76f55725a28cdcaa9b25910f7ac0326a
Successfully built keras-tuner
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162 kB 4.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [29]:
import pandas as pd
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import gc
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, median_absolute_error, explained_variance_score
import numpy as np
import tensorflow as tf
import datetime
import autokeras as ak
from supervised.automl import AutoML
from tensorflow.keras.models import load_model

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [30]:
algos={}

## package for Autopytorch

In [31]:
import  torch
import os
import tempfile as tmp
import warnings

import sklearn.datasets
import sklearn.model_selection

os.environ['JOBLIB_TEMP_FOLDER'] = tmp.gettempdir()
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

from autoPyTorch.api.tabular_regression import TabularRegressionTask

## package for ploating  graph

In [32]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
import csv

# drive Location

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## spec Float

In [34]:
drive_location = r"/content/drive/MyDrive/Colab Notebooks/Internship/SpecFloat/"

## spec int

In [35]:
drive_location = r"/content/drive/MyDrive/Colab Notebooks/Internship/SpecInt/"

## npb

In [36]:
drive_location = r"/content/drive/MyDrive/Colab Notebooks/Internship/AllAlgos/"

# AutoML model function call



In [ ]:
# Mantevo Suite
#benchmark = "Mantevo"
#algo="mantevo_miniFE"
#algo_fname="runtimes_final_mantevo_miniFE"
# NPB Suite
#benchmark = "NPB"
#algo = "npb_ep"
#algo_fname="runtimes_final_npb_ep"
#benchmark = "NPB"
#algo = "npb_mg"
#algo_fname="runtimes_final_npb_mg"
# SPEC 2006 Float
#benchmark = "SPEC2006"
#algo = "leslie3d"
#algo_fname = "437.leslie3d"
# SPEC 2006 Int
#benchmark = "SPEC2006"
#algo = "perlbench"
#algo_fname = "400.perlbench"

spec_float_benchmark_list = ["SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006"]
spec_float_algo_list = ["603.bwaves_s","607.cactuBSSN_s","619.lbm_s","621.wrf_s","627.cam4_s","628.pop2_s","638.imagick_s","644.nab_s","649.fotonik3d_s","654.roms_s","416.gamess","433.milc","434.zeusmp","435.gromacs","437.leslie3d","447.dealII","450.soplex","453.povray","454.calculix"]
spec_float_algo_fname_list = ["603.bwaves_s","607.cactuBSSN_s","619.lbm_s","621.wrf_s","627.cam4_s","628.pop2_s","638.imagick_s","644.nab_s","649.fotonik3d_s","654.roms_s","416.gamess","433.milc","434.zeusmp","435.gromacs","437.leslie3d","447.dealII","450.soplex","453.povray","454.calculix"]

spec_int_benchmark_list = ["SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006"]
spec_int_algo_list = ["600.perlbench_s","602.gcc_s","605.mcf_s","620.omnetpp_s","623.xalancbmk_s","625.x264_s","631.deepsjeng_s","641.leela_s","648.exchange2_s","657.xz_s","401.bzip2","456.hmmer","458.sjeng","462.libquantum","473.astar"]
spec_int_algo_fname_list = ["600.perlbench_s","602.gcc_s","605.mcf_s","620.omnetpp_s","623.xalancbmk_s","625.x264_s","631.deepsjeng_s","641.leela_s","648.exchange2_s","657.xz_s","401.bzip2","456.hmmer","458.sjeng","462.libquantum","473.astar"]

npb_other_benchmark_list = ["NPB","NPB","NPB","NPB","OTHER","OTHER","OTHER"]
npb_other_algo_list = ["npb_ep","npb_mg","npb_sp","npb_sp-mz","matmul","montecarlo","quicksort"]
npb_other_algo_fname_list = ["npb_ep","npb_mg","npb_sp","npb_sp-mz","matmul","montecarlo","quicksort"]



In [ ]:
# Assign variable with which list is to be processed
benchmark_list = spec_float_benchmark_list
algo_list = spec_float_algo_list
algo_fname_list = spec_float_algo_fname_list

In [ ]:
# Assign variable with which list is to be processed
benchmark_list = spec_int_benchmark_list
algo_list = spec_int_algo_list
algo_fname_list = spec_int_algo_fname_list

In [ ]:
# Assign variable with which list is to be processed
benchmark_list = npb_other_benchmark_list
algo_list = npb_other_algo_list
algo_fname_list = npb_other_algo_fname_list

In [ ]:
for algo_name in algo_list:
  algos[algo_name]={}

### time data

#### data

In [ ]:
algos = {'npb_ep': {'nn_start_time': datetime.datetime(2022, 6, 28, 8, 8, 59, 73052), 'nn_end_time': datetime.datetime(2022, 6, 28, 8, 9, 8, 643781), 'nn_total_time': datetime.timedelta(seconds=9, microseconds=570729), 'autoKeras_start_time': datetime.datetime(2022, 6, 28, 8, 12, 9, 740446), 'autoKeras_end_time': datetime.datetime(2022, 6, 28, 8, 13, 12, 629780), 'autoKeras_total_time': datetime.timedelta(seconds=62, microseconds=889334), 'mean_absolute_error_start_time': datetime.datetime(2022, 6, 28, 8, 26, 9, 327839), 'mean_absolute_error_end_time': datetime.datetime(2022, 6, 28, 8, 31, 26, 828293), 'mean_absolute_error_total_time': datetime.timedelta(seconds=317, microseconds=500454), 'mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 9, 3, 33, 761797), 'mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 9, 8, 53, 459305), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=319, microseconds=697508), 'root_mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 10, 11, 16, 98947), 'root_mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 10, 16, 27, 853962), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=311, microseconds=755015), 'median_absolute_error_start_time': datetime.datetime(2022, 6, 28, 10, 52, 39, 183828), 'median_absolute_error_end_time': datetime.datetime(2022, 6, 28, 10, 57, 56, 895746), 'median_absolute_error_total_time': datetime.timedelta(seconds=317, microseconds=711918), 'r2_start_time': datetime.datetime(2022, 6, 28, 11, 46, 10, 343094), 'r2_end_time': datetime.datetime(2022, 6, 28, 11, 51, 26, 45353), 'r2_total_time': datetime.timedelta(seconds=315, microseconds=702259), 'autoMljar_start_time': datetime.datetime(2022, 7, 8, 4, 51, 59, 507480), 'autoMljar_end_time': datetime.datetime(2022, 7, 8, 4, 52, 36, 727499), 'autoMljar_total_time': datetime.timedelta(seconds=37, microseconds=220019)}, 'npb_mg': {'nn_start_time': datetime.datetime(2022, 6, 28, 8, 9, 9, 86252), 'nn_end_time': datetime.datetime(2022, 6, 28, 8, 9, 21, 1751), 'nn_total_time': datetime.timedelta(seconds=11, microseconds=915499), 'autoKeras_start_time': datetime.datetime(2022, 6, 28, 8, 13, 12, 629803), 'autoKeras_end_time': datetime.datetime(2022, 6, 28, 8, 14, 2, 903540), 'autoKeras_total_time': datetime.timedelta(seconds=50, microseconds=273737), 'mean_absolute_error_start_time': datetime.datetime(2022, 6, 28, 8, 31, 26, 828323), 'mean_absolute_error_end_time': datetime.datetime(2022, 6, 28, 8, 36, 41, 205421), 'mean_absolute_error_total_time': datetime.timedelta(seconds=314, microseconds=377098), 'mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 9, 8, 53, 459328), 'mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 9, 14, 3, 305147), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=309, microseconds=845819), 'root_mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 10, 16, 27, 854001), 'root_mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 10, 21, 39, 102936), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=311, microseconds=248935), 'median_absolute_error_start_time': datetime.datetime(2022, 6, 28, 10, 57, 56, 895767), 'median_absolute_error_end_time': datetime.datetime(2022, 6, 28, 11, 3, 7, 532722), 'median_absolute_error_total_time': datetime.timedelta(seconds=310, microseconds=636955), 'r2_start_time': datetime.datetime(2022, 6, 28, 11, 51, 26, 45375), 'r2_end_time': datetime.datetime(2022, 6, 28, 11, 56, 37, 693011), 'r2_total_time': datetime.timedelta(seconds=311, microseconds=647636), 'autoMljar_start_time': datetime.datetime(2022, 7, 8, 4, 52, 36, 727516), 'autoMljar_end_time': datetime.datetime(2022, 7, 8, 4, 53, 7, 162863), 'autoMljar_total_time': datetime.timedelta(seconds=30, microseconds=435347)}, 'npb_sp': {'nn_start_time': datetime.datetime(2022, 6, 28, 8, 9, 21, 459354), 'nn_end_time': datetime.datetime(2022, 6, 28, 8, 9, 37, 384813), 'nn_total_time': datetime.timedelta(seconds=15, microseconds=925459), 'autoKeras_start_time': datetime.datetime(2022, 6, 28, 8, 14, 2, 903562), 'autoKeras_end_time': datetime.datetime(2022, 6, 28, 8, 15, 35, 93058), 'autoKeras_total_time': datetime.timedelta(seconds=92, microseconds=189496), 'mean_absolute_error_start_time': datetime.datetime(2022, 6, 28, 8, 36, 41, 205454), 'mean_absolute_error_end_time': datetime.datetime(2022, 6, 28, 8, 41, 53, 996375), 'mean_absolute_error_total_time': datetime.timedelta(seconds=312, microseconds=790921), 'mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 9, 14, 3, 305176), 'mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 9, 19, 11, 561583), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=308, microseconds=256407), 'root_mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 10, 21, 39, 102961), 'root_mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 10, 26, 49, 88716), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=309, microseconds=985755), 'median_absolute_error_start_time': datetime.datetime(2022, 6, 28, 11, 3, 7, 532747), 'median_absolute_error_end_time': datetime.datetime(2022, 6, 28, 11, 8, 21, 834201), 'median_absolute_error_total_time': datetime.timedelta(seconds=314, microseconds=301454), 'r2_start_time': datetime.datetime(2022, 6, 28, 11, 56, 37, 693036), 'r2_end_time': datetime.datetime(2022, 6, 28, 12, 1, 55, 206325), 'r2_total_time': datetime.timedelta(seconds=317, microseconds=513289), 'autoMljar_start_time': datetime.datetime(2022, 7, 8, 4, 53, 7, 162882), 'autoMljar_end_time': datetime.datetime(2022, 7, 8, 4, 53, 37, 402005), 'autoMljar_total_time': datetime.timedelta(seconds=30, microseconds=239123)}, 'npb_sp-mz': {'nn_start_time': datetime.datetime(2022, 6, 28, 8, 9, 37, 818487), 'nn_end_time': datetime.datetime(2022, 6, 28, 8, 9, 54, 447084), 'nn_total_time': datetime.timedelta(seconds=16, microseconds=628597), 'autoKeras_start_time': datetime.datetime(2022, 6, 28, 8, 15, 35, 93076), 'autoKeras_end_time': datetime.datetime(2022, 6, 28, 8, 17, 5, 192655), 'autoKeras_total_time': datetime.timedelta(seconds=90, microseconds=99579), 'mean_absolute_error_start_time': datetime.datetime(2022, 6, 28, 8, 41, 53, 996400), 'mean_absolute_error_end_time': datetime.datetime(2022, 6, 28, 8, 47, 5, 787124), 'mean_absolute_error_total_time': datetime.timedelta(seconds=311, microseconds=790724), 'mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 9, 19, 11, 561603), 'mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 9, 24, 21, 698836), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=310, microseconds=137233), 'root_mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 10, 26, 49, 88740), 'root_mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 10, 31, 59, 225843), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=310, microseconds=137103), 'median_absolute_error_start_time': datetime.datetime(2022, 6, 28, 11, 8, 21, 834230), 'median_absolute_error_end_time': datetime.datetime(2022, 6, 28, 11, 13, 39, 504551), 'median_absolute_error_total_time': datetime.timedelta(seconds=317, microseconds=670321), 'r2_start_time': datetime.datetime(2022, 6, 28, 12, 1, 55, 206345), 'r2_end_time': datetime.datetime(2022, 6, 28, 12, 7, 13, 487725), 'r2_total_time': datetime.timedelta(seconds=318, microseconds=281380), 'autoMljar_start_time': datetime.datetime(2022, 7, 8, 4, 53, 37, 402026), 'autoMljar_end_time': datetime.datetime(2022, 7, 8, 4, 54, 11, 27591), 'autoMljar_total_time': datetime.timedelta(seconds=33, microseconds=625565)}, 'matmul': {'nn_start_time': datetime.datetime(2022, 6, 28, 8, 9, 55, 12610), 'nn_end_time': datetime.datetime(2022, 6, 28, 8, 10, 5, 469516), 'nn_total_time': datetime.timedelta(seconds=10, microseconds=456906), 'autoKeras_start_time': datetime.datetime(2022, 6, 28, 8, 17, 5, 192679), 'autoKeras_end_time': datetime.datetime(2022, 6, 28, 8, 18, 16, 491847), 'autoKeras_total_time': datetime.timedelta(seconds=71, microseconds=299168), 'mean_absolute_error_start_time': datetime.datetime(2022, 6, 28, 8, 47, 5, 787149), 'mean_absolute_error_end_time': datetime.datetime(2022, 6, 28, 8, 52, 18, 852064), 'mean_absolute_error_total_time': datetime.timedelta(seconds=313, microseconds=64915), 'mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 9, 24, 21, 698859), 'mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 9, 29, 34, 543482), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=312, microseconds=844623), 'root_mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 10, 31, 59, 225875), 'root_mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 10, 37, 11, 100172), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=311, microseconds=874297), 'median_absolute_error_start_time': datetime.datetime(2022, 6, 28, 11, 13, 39, 504581), 'median_absolute_error_end_time': datetime.datetime(2022, 6, 28, 11, 18, 50, 171776), 'median_absolute_error_total_time': datetime.timedelta(seconds=310, microseconds=667195), 'r2_start_time': datetime.datetime(2022, 6, 28, 12, 7, 13, 487746), 'r2_end_time': datetime.datetime(2022, 6, 28, 12, 12, 23, 467752), 'r2_total_time': datetime.timedelta(seconds=309, microseconds=980006), 'autoMljar_start_time': datetime.datetime(2022, 7, 8, 4, 54, 11, 27611), 'autoMljar_end_time': datetime.datetime(2022, 7, 8, 4, 54, 49, 27267), 'autoMljar_total_time': datetime.timedelta(seconds=37, microseconds=999656)}, 'montecarlo': {'nn_start_time': datetime.datetime(2022, 6, 28, 8, 10, 6, 43268), 'nn_end_time': datetime.datetime(2022, 6, 28, 8, 10, 14, 935894), 'nn_total_time': datetime.timedelta(seconds=8, microseconds=892626), 'autoKeras_start_time': datetime.datetime(2022, 6, 28, 8, 18, 16, 491872), 'autoKeras_end_time': datetime.datetime(2022, 6, 28, 8, 19, 23, 638412), 'autoKeras_total_time': datetime.timedelta(seconds=67, microseconds=146540), 'mean_absolute_error_start_time': datetime.datetime(2022, 6, 28, 8, 52, 18, 852091), 'mean_absolute_error_end_time': datetime.datetime(2022, 6, 28, 8, 57, 41, 549711), 'mean_absolute_error_total_time': datetime.timedelta(seconds=322, microseconds=697620), 'mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 9, 29, 34, 543509), 'mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 9, 34, 43, 672050), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=309, microseconds=128541), 'root_mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 10, 37, 11, 100331), 'root_mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 10, 42, 23, 302245), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=312, microseconds=201914), 'median_absolute_error_start_time': datetime.datetime(2022, 6, 28, 11, 18, 50, 171803), 'median_absolute_error_end_time': datetime.datetime(2022, 6, 28, 11, 24, 2, 772627), 'median_absolute_error_total_time': datetime.timedelta(seconds=312, microseconds=600824), 'r2_start_time': datetime.datetime(2022, 6, 28, 12, 12, 23, 467780), 'r2_end_time': datetime.datetime(2022, 6, 28, 12, 17, 35, 674741), 'r2_total_time': datetime.timedelta(seconds=312, microseconds=206961), 'autoMljar_start_time': datetime.datetime(2022, 7, 8, 4, 54, 49, 27287), 'autoMljar_end_time': datetime.datetime(2022, 7, 8, 4, 55, 26, 477863), 'autoMljar_total_time': datetime.timedelta(seconds=37, microseconds=450576)}, 'quicksort': {'nn_start_time': datetime.datetime(2022, 6, 28, 8, 10, 15, 686791), 'nn_end_time': datetime.datetime(2022, 6, 28, 8, 10, 40, 80699), 'nn_total_time': datetime.timedelta(seconds=24, microseconds=393908), 'autoKeras_start_time': datetime.datetime(2022, 6, 28, 8, 19, 23, 638436), 'autoKeras_end_time': datetime.datetime(2022, 6, 28, 8, 20, 25, 716849), 'autoKeras_total_time': datetime.timedelta(seconds=62, microseconds=78413), 'mean_absolute_error_start_time': datetime.datetime(2022, 6, 28, 8, 57, 41, 549736), 'mean_absolute_error_end_time': datetime.datetime(2022, 6, 28, 9, 2, 53, 129897), 'mean_absolute_error_total_time': datetime.timedelta(seconds=311, microseconds=580161), 'mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 9, 34, 43, 672075), 'mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 9, 39, 53, 833931), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=310, microseconds=161856), 'root_mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 10, 42, 23, 302269), 'root_mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 10, 47, 35, 516790), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=312, microseconds=214521), 'median_absolute_error_start_time': datetime.datetime(2022, 6, 28, 11, 24, 2, 772653), 'median_absolute_error_end_time': datetime.datetime(2022, 6, 28, 11, 29, 12, 675505), 'median_absolute_error_total_time': datetime.timedelta(seconds=309, microseconds=902852), 'r2_start_time': datetime.datetime(2022, 6, 28, 12, 17, 35, 674765), 'r2_end_time': datetime.datetime(2022, 6, 28, 12, 22, 46, 111484), 'r2_total_time': datetime.timedelta(seconds=310, microseconds=436719), 'autoMljar_start_time': datetime.datetime(2022, 7, 8, 4, 55, 26, 477883), 'autoMljar_end_time': datetime.datetime(2022, 7, 8, 4, 56, 5, 843551), 'autoMljar_total_time': datetime.timedelta(seconds=39, microseconds=365668)}, '600.perlbench_s': {'nn_start_time': datetime.datetime(2022, 7, 5, 13, 12, 8, 523469), 'nn_end_time': datetime.datetime(2022, 7, 5, 13, 13, 40, 954722), 'nn_total_time': datetime.timedelta(seconds=92, microseconds=431253), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 13, 13, 40, 954765), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 13, 16, 12, 616345), 'autoKeras_total_time': datetime.timedelta(seconds=151, microseconds=661580), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 4, 52, 40, 951496), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 4, 57, 55, 896029), 'mean_absolute_error_total_time': datetime.timedelta(seconds=314, microseconds=944533), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 6, 24, 6, 322440), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 6, 29, 24, 334155), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=318, microseconds=11715), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 13, 34, 27, 434592), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 13, 39, 37, 351128), 'median_absolute_error_total_time': datetime.timedelta(seconds=309, microseconds=916536), 'r2_start_time': datetime.datetime(2022, 7, 6, 15, 52, 19, 168821), 'r2_end_time': datetime.datetime(2022, 7, 6, 15, 57, 44, 264524), 'r2_total_time': datetime.timedelta(seconds=325, microseconds=95703), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 38, 19, 619129), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 39, 1, 836508), 'autoMljar_total_time': datetime.timedelta(seconds=42, microseconds=217379), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 8, 49, 828632), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 14, 11, 743075), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=321, microseconds=914443)}, '602.gcc_s': {'nn_start_time': datetime.datetime(2022, 7, 5, 13, 16, 13, 330613), 'nn_end_time': datetime.datetime(2022, 7, 5, 13, 17, 50, 212813), 'nn_total_time': datetime.timedelta(seconds=96, microseconds=882200), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 13, 17, 50, 212826), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 13, 20, 22, 290561), 'autoKeras_total_time': datetime.timedelta(seconds=152, microseconds=77735), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 4, 57, 56, 386251), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 5, 3, 10, 486108), 'mean_absolute_error_total_time': datetime.timedelta(seconds=314, microseconds=99857), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 6, 29, 24, 452218), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 6, 34, 38, 318377), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=313, microseconds=866159), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 13, 39, 38, 428808), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 13, 44, 59, 782869), 'median_absolute_error_total_time': datetime.timedelta(seconds=321, microseconds=354061), 'r2_start_time': datetime.datetime(2022, 7, 6, 15, 57, 44, 382786), 'r2_end_time': datetime.datetime(2022, 7, 6, 16, 3, 1, 808592), 'r2_total_time': datetime.timedelta(seconds=317, microseconds=425806), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 39, 1, 836533), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 39, 42, 391200), 'autoMljar_total_time': datetime.timedelta(seconds=40, microseconds=554667), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 14, 11, 743101), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 19, 31, 333734), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=319, microseconds=590633)}, '605.mcf_s': {'nn_start_time': datetime.datetime(2022, 7, 5, 13, 20, 22, 848836), 'nn_end_time': datetime.datetime(2022, 7, 5, 13, 22, 8, 151721), 'nn_total_time': datetime.timedelta(seconds=105, microseconds=302885), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 13, 22, 8, 151738), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 13, 24, 44, 52544), 'autoKeras_total_time': datetime.timedelta(seconds=155, microseconds=900806), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 5, 3, 11, 186752), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 5, 8, 25, 707316), 'mean_absolute_error_total_time': datetime.timedelta(seconds=314, microseconds=520564), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 6, 34, 38, 446959), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 6, 39, 49, 668482), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=311, microseconds=221523), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 13, 45, 0, 867129), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 13, 50, 15, 948499), 'median_absolute_error_total_time': datetime.timedelta(seconds=315, microseconds=81370), 'r2_start_time': datetime.datetime(2022, 7, 6, 16, 3, 1, 943010), 'r2_end_time': datetime.datetime(2022, 7, 6, 16, 8, 19, 191063), 'r2_total_time': datetime.timedelta(seconds=317, microseconds=248053), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 39, 42, 391215), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 40, 21, 563954), 'autoMljar_total_time': datetime.timedelta(seconds=39, microseconds=172739), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 19, 31, 333754), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 24, 43, 716858), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=312, microseconds=383104)}, '620.omnetpp_s': {'nn_start_time': datetime.datetime(2022, 7, 5, 13, 24, 44, 606441), 'nn_end_time': datetime.datetime(2022, 7, 5, 13, 25, 46, 986471), 'nn_total_time': datetime.timedelta(seconds=62, microseconds=380030), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 13, 25, 46, 986486), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 13, 28, 21, 5664), 'autoKeras_total_time': datetime.timedelta(seconds=154, microseconds=19178), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 5, 8, 26, 336980), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 5, 13, 50, 177903), 'mean_absolute_error_total_time': datetime.timedelta(seconds=323, microseconds=840923), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 6, 39, 49, 806928), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 6, 45, 1, 816702), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=312, microseconds=9774), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 13, 50, 17, 23075), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 13, 55, 37, 741919), 'median_absolute_error_total_time': datetime.timedelta(seconds=320, microseconds=718844), 'r2_start_time': datetime.datetime(2022, 7, 6, 16, 8, 19, 324052), 'r2_end_time': datetime.datetime(2022, 7, 6, 16, 13, 33, 281002), 'r2_total_time': datetime.timedelta(seconds=313, microseconds=956950), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 40, 21, 563971), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 41, 2, 666326), 'autoMljar_total_time': datetime.timedelta(seconds=41, microseconds=102355), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 24, 43, 716878), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 30, 2, 588857), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=318, microseconds=871979)}, '623.xalancbmk_s': {'nn_start_time': datetime.datetime(2022, 7, 5, 13, 28, 21, 505349), 'nn_end_time': datetime.datetime(2022, 7, 5, 13, 30, 17, 316035), 'nn_total_time': datetime.timedelta(seconds=115, microseconds=810686), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 13, 30, 17, 316055), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 13, 32, 53, 769906), 'autoKeras_total_time': datetime.timedelta(seconds=156, microseconds=453851), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 5, 13, 50, 759361), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 5, 19, 3, 800188), 'mean_absolute_error_total_time': datetime.timedelta(seconds=313, microseconds=40827), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 6, 45, 1, 966155), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 6, 50, 14, 346192), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=312, microseconds=380037), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 13, 55, 38, 799175), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 14, 0, 59, 947067), 'median_absolute_error_total_time': datetime.timedelta(seconds=321, microseconds=147892), 'r2_start_time': datetime.datetime(2022, 7, 6, 16, 13, 33, 396832), 'r2_end_time': datetime.datetime(2022, 7, 6, 16, 18, 44, 417290), 'r2_total_time': datetime.timedelta(seconds=311, microseconds=20458), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 41, 2, 666346), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 41, 40, 897308), 'autoMljar_total_time': datetime.timedelta(seconds=38, microseconds=230962), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 30, 2, 588875), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 35, 19, 435745), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=316, microseconds=846870)}, '625.x264_s': {'nn_start_time': datetime.datetime(2022, 7, 5, 13, 32, 54, 292755), 'nn_end_time': datetime.datetime(2022, 7, 5, 13, 33, 58, 143183), 'nn_total_time': datetime.timedelta(seconds=63, microseconds=850428), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 13, 33, 58, 143197), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 13, 35, 30, 60066), 'autoKeras_total_time': datetime.timedelta(seconds=91, microseconds=916869), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 5, 19, 4, 416255), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 5, 24, 24, 357566), 'mean_absolute_error_total_time': datetime.timedelta(seconds=319, microseconds=941311), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 6, 50, 14, 499740), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 6, 55, 37, 822860), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=323, microseconds=323120), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 14, 1, 1, 113112), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 14, 6, 20, 733927), 'median_absolute_error_total_time': datetime.timedelta(seconds=319, microseconds=620815), 'r2_start_time': datetime.datetime(2022, 7, 6, 16, 18, 44, 547875), 'r2_end_time': datetime.datetime(2022, 7, 6, 16, 23, 56, 899541), 'r2_total_time': datetime.timedelta(seconds=312, microseconds=351666), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 41, 40, 897327), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 42, 20, 901694), 'autoMljar_total_time': datetime.timedelta(seconds=40, microseconds=4367), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 35, 19, 435768), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 40, 39, 709389), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=320, microseconds=273621)}, '631.deepsjeng_s': {'nn_start_time': datetime.datetime(2022, 7, 5, 13, 35, 30, 585365), 'nn_end_time': datetime.datetime(2022, 7, 5, 13, 37, 6, 994637), 'nn_total_time': datetime.timedelta(seconds=96, microseconds=409272), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 13, 37, 6, 994650), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 13, 38, 39, 893470), 'autoKeras_total_time': datetime.timedelta(seconds=92, microseconds=898820), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 5, 24, 24, 963597), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 5, 29, 38, 296703), 'mean_absolute_error_total_time': datetime.timedelta(seconds=313, microseconds=333106), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 6, 55, 37, 959033), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 7, 0, 57, 298457), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=319, microseconds=339424), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 14, 6, 21, 841714), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 14, 11, 42, 344138), 'median_absolute_error_total_time': datetime.timedelta(seconds=320, microseconds=502424), 'r2_start_time': datetime.datetime(2022, 7, 6, 16, 23, 57, 43395), 'r2_end_time': datetime.datetime(2022, 7, 6, 16, 29, 11, 40878), 'r2_total_time': datetime.timedelta(seconds=313, microseconds=997483), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 42, 20, 901711), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 43, 1, 321162), 'autoMljar_total_time': datetime.timedelta(seconds=40, microseconds=419451), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 40, 39, 709409), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 46, 3, 682559), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=323, microseconds=973150)}, '641.leela_s': {'nn_start_time': datetime.datetime(2022, 7, 5, 13, 38, 40, 397343), 'nn_end_time': datetime.datetime(2022, 7, 5, 13, 39, 57, 611486), 'nn_total_time': datetime.timedelta(seconds=77, microseconds=214143), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 13, 39, 57, 611500), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 13, 42, 31, 810035), 'autoKeras_total_time': datetime.timedelta(seconds=154, microseconds=198535), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 5, 29, 38, 807931), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 5, 34, 55, 348953), 'mean_absolute_error_total_time': datetime.timedelta(seconds=316, microseconds=541022), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 7, 0, 57, 438581), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 7, 6, 13, 221805), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=315, microseconds=783224), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 14, 11, 43, 454900), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 14, 17, 4, 508243), 'median_absolute_error_total_time': datetime.timedelta(seconds=321, microseconds=53343), 'r2_start_time': datetime.datetime(2022, 7, 6, 16, 29, 11, 172799), 'r2_end_time': datetime.datetime(2022, 7, 6, 16, 34, 24, 206912), 'r2_total_time': datetime.timedelta(seconds=313, microseconds=34113), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 43, 1, 321178), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 43, 46, 647359), 'autoMljar_total_time': datetime.timedelta(seconds=45, microseconds=326181), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 46, 3, 682577), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 51, 24, 176185), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=320, microseconds=493608)}, '648.exchange2_s': {'nn_start_time': datetime.datetime(2022, 7, 5, 13, 42, 32, 360617), 'nn_end_time': datetime.datetime(2022, 7, 5, 13, 43, 55, 504092), 'nn_total_time': datetime.timedelta(seconds=83, microseconds=143475), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 13, 43, 55, 504105), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 13, 46, 30, 29423), 'autoKeras_total_time': datetime.timedelta(seconds=154, microseconds=525318), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 5, 34, 56, 24172), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 5, 40, 9, 228085), 'mean_absolute_error_total_time': datetime.timedelta(seconds=313, microseconds=203913), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 7, 6, 13, 371069), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 7, 11, 28, 904552), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=315, microseconds=533483), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 14, 17, 5, 613164), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 14, 22, 25, 662767), 'median_absolute_error_total_time': datetime.timedelta(seconds=320, microseconds=49603), 'r2_start_time': datetime.datetime(2022, 7, 6, 16, 34, 24, 354764), 'r2_end_time': datetime.datetime(2022, 7, 6, 16, 39, 36, 550372), 'r2_total_time': datetime.timedelta(seconds=312, microseconds=195608), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 43, 46, 647374), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 44, 30, 661615), 'autoMljar_total_time': datetime.timedelta(seconds=44, microseconds=14241), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 51, 24, 176201), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 56, 45, 642040), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=321, microseconds=465839)}, '657.xz_s': {'nn_start_time': datetime.datetime(2022, 7, 5, 13, 46, 30, 570606), 'nn_end_time': datetime.datetime(2022, 7, 5, 13, 48, 20, 327105), 'nn_total_time': datetime.timedelta(seconds=109, microseconds=756499), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 13, 48, 20, 327118), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 13, 50, 24, 59926), 'autoKeras_total_time': datetime.timedelta(seconds=123, microseconds=732808), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 5, 40, 9, 782611), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 5, 45, 32, 732048), 'mean_absolute_error_total_time': datetime.timedelta(seconds=322, microseconds=949437), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 7, 11, 29, 21908), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 7, 16, 47, 745029), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=318, microseconds=723121), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 14, 22, 26, 782676), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 14, 27, 47, 912628), 'median_absolute_error_total_time': datetime.timedelta(seconds=321, microseconds=129952), 'r2_start_time': datetime.datetime(2022, 7, 6, 16, 39, 36, 675238), 'r2_end_time': datetime.datetime(2022, 7, 6, 16, 44, 54, 438912), 'r2_total_time': datetime.timedelta(seconds=317, microseconds=763674), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 44, 30, 661634), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 45, 19, 846712), 'autoMljar_total_time': datetime.timedelta(seconds=49, microseconds=185078), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 56, 45, 642056), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 14, 1, 59, 23944), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=313, microseconds=381888)}, '401.bzip2': {'nn_start_time': datetime.datetime(2022, 7, 5, 13, 50, 24, 601590), 'nn_end_time': datetime.datetime(2022, 7, 5, 13, 51, 31, 979070), 'nn_total_time': datetime.timedelta(seconds=67, microseconds=377480), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 13, 51, 31, 979083), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 13, 53, 42, 918084), 'autoKeras_total_time': datetime.timedelta(seconds=130, microseconds=939001), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 5, 45, 33, 288628), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 5, 50, 50, 669639), 'mean_absolute_error_total_time': datetime.timedelta(seconds=317, microseconds=381011), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 7, 16, 47, 880642), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 7, 21, 57, 62182), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=309, microseconds=181540), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 14, 27, 49, 50754), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 14, 32, 59, 696449), 'median_absolute_error_total_time': datetime.timedelta(seconds=310, microseconds=645695), 'r2_start_time': datetime.datetime(2022, 7, 6, 16, 44, 54, 580572), 'r2_end_time': datetime.datetime(2022, 7, 6, 16, 50, 12, 479570), 'r2_total_time': datetime.timedelta(seconds=317, microseconds=898998), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 45, 19, 846729), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 46, 18, 708580), 'autoMljar_total_time': datetime.timedelta(seconds=58, microseconds=861851), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 14, 1, 59, 23963), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 14, 7, 22, 864066), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=323, microseconds=840103)}, '456.hmmer': {'nn_start_time': datetime.datetime(2022, 7, 5, 13, 53, 43, 501202), 'nn_end_time': datetime.datetime(2022, 7, 5, 13, 55, 12, 358861), 'nn_total_time': datetime.timedelta(seconds=88, microseconds=857659), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 13, 55, 12, 358880), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 13, 57, 40, 811018), 'autoKeras_total_time': datetime.timedelta(seconds=148, microseconds=452138), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 5, 50, 51, 164828), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 5, 56, 10, 413512), 'mean_absolute_error_total_time': datetime.timedelta(seconds=319, microseconds=248684), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 7, 21, 57, 194516), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 7, 27, 17, 436225), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=320, microseconds=241709), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 14, 33, 0, 855572), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 14, 38, 14, 241333), 'median_absolute_error_total_time': datetime.timedelta(seconds=313, microseconds=385761), 'r2_start_time': datetime.datetime(2022, 7, 6, 16, 50, 12, 614383), 'r2_end_time': datetime.datetime(2022, 7, 6, 16, 55, 30, 265131), 'r2_total_time': datetime.timedelta(seconds=317, microseconds=650748), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 46, 18, 708599), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 47, 19, 424282), 'autoMljar_total_time': datetime.timedelta(seconds=60, microseconds=715683), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 14, 7, 22, 864085), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 14, 12, 37, 362091), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=314, microseconds=498006)}, '458.sjeng': {'nn_start_time': datetime.datetime(2022, 7, 5, 13, 57, 41, 522578), 'nn_end_time': datetime.datetime(2022, 7, 5, 13, 58, 28, 8552), 'nn_total_time': datetime.timedelta(seconds=46, microseconds=485974), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 13, 58, 28, 8564), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 14, 0, 51, 309433), 'autoKeras_total_time': datetime.timedelta(seconds=143, microseconds=300869), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 5, 56, 10, 973632), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 6, 1, 30, 559525), 'mean_absolute_error_total_time': datetime.timedelta(seconds=319, microseconds=585893), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 7, 27, 17, 583968), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 7, 32, 26, 667996), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=309, microseconds=84028), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 14, 38, 15, 366635), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 14, 43, 31, 160117), 'median_absolute_error_total_time': datetime.timedelta(seconds=315, microseconds=793482), 'r2_start_time': datetime.datetime(2022, 7, 6, 16, 55, 30, 416753), 'r2_end_time': datetime.datetime(2022, 7, 6, 17, 0, 46, 123430), 'r2_total_time': datetime.timedelta(seconds=315, microseconds=706677), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 47, 19, 424298), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 48, 19, 723070), 'autoMljar_total_time': datetime.timedelta(seconds=60, microseconds=298772), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 14, 12, 37, 362109), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 14, 17, 52, 542784), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=315, microseconds=180675)}, '462.libquantum': {'nn_start_time': datetime.datetime(2022, 7, 5, 14, 0, 51, 856869), 'nn_end_time': datetime.datetime(2022, 7, 5, 14, 1, 50, 546012), 'nn_total_time': datetime.timedelta(seconds=58, microseconds=689143), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 14, 1, 50, 546029), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 14, 4, 28, 156337), 'autoKeras_total_time': datetime.timedelta(seconds=157, microseconds=610308), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 6, 1, 31, 111857), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 6, 6, 53, 384293), 'mean_absolute_error_total_time': datetime.timedelta(seconds=322, microseconds=272436), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 7, 32, 26, 806663), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 7, 37, 35, 538135), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=308, microseconds=731472), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 14, 43, 32, 291233), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 14, 48, 47, 941192), 'median_absolute_error_total_time': datetime.timedelta(seconds=315, microseconds=649959), 'r2_start_time': datetime.datetime(2022, 7, 6, 17, 0, 46, 270285), 'r2_end_time': datetime.datetime(2022, 7, 6, 17, 6, 9, 158781), 'r2_total_time': datetime.timedelta(seconds=322, microseconds=888496), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 48, 19, 723090), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 49, 20, 180392), 'autoMljar_total_time': datetime.timedelta(seconds=60, microseconds=457302), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 14, 17, 52, 542804), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 14, 23, 15, 907216), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=323, microseconds=364412)}, '473.astar': {'nn_start_time': datetime.datetime(2022, 7, 5, 14, 4, 28, 738394), 'nn_end_time': datetime.datetime(2022, 7, 5, 14, 5, 40, 855749), 'nn_total_time': datetime.timedelta(seconds=72, microseconds=117355), 'autoKeras_start_time': datetime.datetime(2022, 7, 5, 14, 5, 40, 855764), 'autoKeras_end_time': datetime.datetime(2022, 7, 5, 14, 9, 4, 646308), 'autoKeras_total_time': datetime.timedelta(seconds=203, microseconds=790544), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 6, 6, 6, 54, 48805), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 6, 6, 12, 12, 645744), 'mean_absolute_error_total_time': datetime.timedelta(seconds=318, microseconds=596939), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 6, 7, 37, 35, 668176), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 6, 7, 42, 56, 100459), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=320, microseconds=432283), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 6, 14, 48, 49, 110671), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 6, 14, 54, 1, 829599), 'median_absolute_error_total_time': datetime.timedelta(seconds=312, microseconds=718928), 'r2_start_time': datetime.datetime(2022, 7, 6, 17, 6, 9, 314894), 'r2_end_time': datetime.datetime(2022, 7, 6, 17, 11, 22, 176552), 'r2_total_time': datetime.timedelta(seconds=312, microseconds=861658), 'autoMljar_start_time': datetime.datetime(2022, 7, 6, 17, 49, 20, 180406), 'autoMljar_end_time': datetime.datetime(2022, 7, 6, 17, 50, 30, 631652), 'autoMljar_total_time': datetime.timedelta(seconds=70, microseconds=451246), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 14, 23, 15, 907234), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 14, 28, 38, 843503), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=322, microseconds=936269)}, '603.bwaves_s': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 25, 45, 688360), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 26, 33, 900993), 'nn_total_time': datetime.timedelta(seconds=48, microseconds=212633), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 27, 28, 153617), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 11, 18, 436206), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 13, 26, 202243), 'autoKeras_total_time': datetime.timedelta(seconds=127, microseconds=766037), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 10, 23, 36, 379734), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 10, 29, 21, 679454), 'mean_absolute_error_total_time': datetime.timedelta(seconds=345, microseconds=299720), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 15, 47, 24, 353634), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 15, 52, 41, 769670), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=317, microseconds=416036), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 28, 8, 184298), 'autoMljar_total_time': datetime.timedelta(seconds=40, microseconds=30681), 'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 11, 5, 17, 18, 255450), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 5, 18, 14, 37862), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 5, 23, 23, 943237), 'median_absolute_error_total_time': datetime.timedelta(seconds=309, microseconds=905375), 'r2_start_time': datetime.datetime(2022, 7, 11, 12, 9, 51, 604959), 'r2_end_time': datetime.datetime(2022, 7, 11, 12, 15, 3, 96494), 'r2_total_time': datetime.timedelta(seconds=311, microseconds=491535)}, '607.cactuBSSN_s': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 26, 34, 301796), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 27, 28, 922191), 'nn_total_time': datetime.timedelta(seconds=54, microseconds=620395), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 13, 26, 306366), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 15, 44, 190518), 'autoKeras_total_time': datetime.timedelta(seconds=137, microseconds=884152), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 10, 29, 21, 679476), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 10, 34, 42, 954869), 'mean_absolute_error_total_time': datetime.timedelta(seconds=321, microseconds=275393), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 15, 52, 41, 769695), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 15, 57, 52, 950888), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=311, microseconds=181193), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 28, 8, 184312), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 28, 41, 457251), 'autoMljar_total_time': datetime.timedelta(seconds=33, microseconds=272939), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 5, 23, 23, 943252), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 5, 29, 33, 806995), 'median_absolute_error_total_time': datetime.timedelta(seconds=369, microseconds=863743), 'r2_start_time': datetime.datetime(2022, 7, 11, 12, 15, 3, 96509), 'r2_end_time': datetime.datetime(2022, 7, 11, 12, 21, 5, 992669), 'r2_total_time': datetime.timedelta(seconds=362, microseconds=896160)}, '619.lbm_s': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 27, 29, 975010), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 28, 23, 17224), 'nn_total_time': datetime.timedelta(seconds=53, microseconds=42214), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 15, 44, 291466), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 17, 31, 503325), 'autoKeras_total_time': datetime.timedelta(seconds=107, microseconds=211859), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 10, 34, 42, 954891), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 10, 40, 11, 635527), 'mean_absolute_error_total_time': datetime.timedelta(seconds=328, microseconds=680636), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 15, 57, 52, 950907), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 16, 3, 3, 991528), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=311, microseconds=40621), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 28, 41, 457262), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 29, 14, 988601), 'autoMljar_total_time': datetime.timedelta(seconds=33, microseconds=531339), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 5, 29, 33, 807014), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 5, 35, 34, 283468), 'median_absolute_error_total_time': datetime.timedelta(seconds=360, microseconds=476454), 'r2_start_time': datetime.datetime(2022, 7, 11, 12, 21, 5, 992690), 'r2_end_time': datetime.datetime(2022, 7, 11, 12, 27, 11, 317191), 'r2_total_time': datetime.timedelta(seconds=365, microseconds=324501)}, '621.wrf_s': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 28, 23, 392119), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 29, 33, 155756), 'nn_total_time': datetime.timedelta(seconds=69, microseconds=763637), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 17, 31, 626803), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 19, 43, 775765), 'autoKeras_total_time': datetime.timedelta(seconds=132, microseconds=148962), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 10, 40, 11, 635546), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 10, 45, 39, 696330), 'mean_absolute_error_total_time': datetime.timedelta(seconds=328, microseconds=60784), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 16, 3, 3, 991548), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 16, 8, 21, 798841), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=317, microseconds=807293), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 29, 14, 988625), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 29, 50, 695911), 'autoMljar_total_time': datetime.timedelta(seconds=35, microseconds=707286), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 5, 35, 34, 283497), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 5, 41, 42, 69213), 'median_absolute_error_total_time': datetime.timedelta(seconds=367, microseconds=785716), 'r2_start_time': datetime.datetime(2022, 7, 11, 12, 27, 11, 317209), 'r2_end_time': datetime.datetime(2022, 7, 11, 12, 33, 19, 538889), 'r2_total_time': datetime.timedelta(seconds=368, microseconds=221680)}, '627.cam4_s': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 29, 33, 761219), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 30, 26, 718795), 'nn_total_time': datetime.timedelta(seconds=52, microseconds=957576), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 19, 43, 881226), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 21, 36, 668004), 'autoKeras_total_time': datetime.timedelta(seconds=112, microseconds=786778), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 10, 45, 39, 696350), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 10, 51, 13, 307487), 'mean_absolute_error_total_time': datetime.timedelta(seconds=333, microseconds=611137), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 16, 8, 21, 798863), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 16, 13, 41, 93838), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=319, microseconds=294975), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 29, 50, 695923), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 30, 24, 914957), 'autoMljar_total_time': datetime.timedelta(seconds=34, microseconds=219034), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 5, 41, 42, 69231), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 5, 47, 41, 386653), 'median_absolute_error_total_time': datetime.timedelta(seconds=359, microseconds=317422), 'r2_start_time': datetime.datetime(2022, 7, 11, 12, 33, 19, 538908), 'r2_end_time': datetime.datetime(2022, 7, 11, 12, 38, 33, 996065), 'r2_total_time': datetime.timedelta(seconds=314, microseconds=457157)}, '628.pop2_s': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 30, 27, 159161), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 31, 33, 556545), 'nn_total_time': datetime.timedelta(seconds=66, microseconds=397384), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 21, 36, 766993), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 23, 57, 974210), 'autoKeras_total_time': datetime.timedelta(seconds=141, microseconds=207217), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 10, 51, 13, 307507), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 10, 56, 29, 967822), 'mean_absolute_error_total_time': datetime.timedelta(seconds=316, microseconds=660315), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 16, 13, 41, 93857), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 16, 18, 53, 272689), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=312, microseconds=178832), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 30, 24, 914969), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 31, 1, 949345), 'autoMljar_total_time': datetime.timedelta(seconds=37, microseconds=34376), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 5, 47, 41, 386675), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 5, 52, 54, 412343), 'median_absolute_error_total_time': datetime.timedelta(seconds=313, microseconds=25668), 'r2_start_time': datetime.datetime(2022, 7, 11, 12, 38, 33, 996098), 'r2_end_time': datetime.datetime(2022, 7, 11, 12, 43, 49, 503711), 'r2_total_time': datetime.timedelta(seconds=315, microseconds=507613)}, '638.imagick_s': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 31, 33, 964889), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 32, 48, 654121), 'nn_total_time': datetime.timedelta(seconds=74, microseconds=689232), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 23, 58, 93331), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 26, 15, 764568), 'autoKeras_total_time': datetime.timedelta(seconds=137, microseconds=671237), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 10, 56, 29, 967841), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 11, 2, 3, 569180), 'mean_absolute_error_total_time': datetime.timedelta(seconds=333, microseconds=601339), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 16, 18, 53, 272712), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 16, 24, 5, 790110), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=312, microseconds=517398), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 31, 1, 949356), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 31, 39, 186459), 'autoMljar_total_time': datetime.timedelta(seconds=37, microseconds=237103), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 5, 52, 54, 412366), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 5, 58, 4, 647266), 'median_absolute_error_total_time': datetime.timedelta(seconds=310, microseconds=234900), 'r2_start_time': datetime.datetime(2022, 7, 11, 12, 43, 49, 503739), 'r2_end_time': datetime.datetime(2022, 7, 11, 12, 49, 7, 507968), 'r2_total_time': datetime.timedelta(seconds=318, microseconds=4229)}, '644.nab_s': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 32, 49, 47966), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 33, 37, 540931), 'nn_total_time': datetime.timedelta(seconds=48, microseconds=492965), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 26, 15, 867421), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 28, 35, 400268), 'autoKeras_total_time': datetime.timedelta(seconds=139, microseconds=532847), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 11, 2, 3, 569200), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 11, 7, 32, 262442), 'mean_absolute_error_total_time': datetime.timedelta(seconds=328, microseconds=693242), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 16, 24, 5, 790130), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 16, 29, 20, 911773), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=315, microseconds=121643), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 31, 39, 186471), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 32, 29, 417883), 'autoMljar_total_time': datetime.timedelta(seconds=50, microseconds=231412), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 5, 58, 4, 647283), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 6, 3, 24, 655727), 'median_absolute_error_total_time': datetime.timedelta(seconds=320, microseconds=8444), 'r2_start_time': datetime.datetime(2022, 7, 11, 12, 49, 7, 507987), 'r2_end_time': datetime.datetime(2022, 7, 11, 12, 54, 30, 269563), 'r2_total_time': datetime.timedelta(seconds=322, microseconds=761576)}, '649.fotonik3d_s': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 33, 37, 888047), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 34, 10, 247396), 'nn_total_time': datetime.timedelta(seconds=32, microseconds=359349), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 28, 35, 514236), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 30, 46, 46578), 'autoKeras_total_time': datetime.timedelta(seconds=130, microseconds=532342), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 11, 7, 32, 262460), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 11, 12, 49, 956950), 'mean_absolute_error_total_time': datetime.timedelta(seconds=317, microseconds=694490), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 16, 29, 20, 911792), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 16, 34, 35, 861464), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=314, microseconds=949672), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 32, 29, 417894), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 33, 11, 555453), 'autoMljar_total_time': datetime.timedelta(seconds=42, microseconds=137559), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 6, 3, 24, 655747), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 6, 9, 24, 838357), 'median_absolute_error_total_time': datetime.timedelta(seconds=360, microseconds=182610), 'r2_start_time': datetime.datetime(2022, 7, 11, 12, 54, 30, 269579), 'r2_end_time': datetime.datetime(2022, 7, 11, 13, 0, 29, 444490), 'r2_total_time': datetime.timedelta(seconds=359, microseconds=174911)}, '654.roms_s': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 34, 10, 673020), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 34, 38, 694001), 'nn_total_time': datetime.timedelta(seconds=28, microseconds=20981), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 30, 46, 148175), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 32, 44, 828363), 'autoKeras_total_time': datetime.timedelta(seconds=118, microseconds=680188), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 11, 12, 49, 956969), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 11, 18, 8, 220965), 'mean_absolute_error_total_time': datetime.timedelta(seconds=318, microseconds=263996), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 16, 34, 35, 861484), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 16, 39, 51, 912567), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=316, microseconds=51083), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 33, 11, 555467), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 33, 50, 686813), 'autoMljar_total_time': datetime.timedelta(seconds=39, microseconds=131346), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 6, 9, 24, 838376), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 6, 14, 41, 383314), 'median_absolute_error_total_time': datetime.timedelta(seconds=316, microseconds=544938), 'r2_start_time': datetime.datetime(2022, 7, 11, 13, 0, 29, 444509), 'r2_end_time': datetime.datetime(2022, 7, 11, 13, 5, 42, 854133), 'r2_total_time': datetime.timedelta(seconds=313, microseconds=409624)}, '416.gamess': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 34, 39, 110732), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 35, 26, 220581), 'nn_total_time': datetime.timedelta(seconds=47, microseconds=109849), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 32, 44, 942979), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 34, 39, 379970), 'autoKeras_total_time': datetime.timedelta(seconds=114, microseconds=436991), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 11, 18, 8, 220982), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 11, 23, 38, 540159), 'mean_absolute_error_total_time': datetime.timedelta(seconds=330, microseconds=319177), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 16, 39, 51, 912584), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 16, 45, 1, 986028), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=310, microseconds=73444), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 33, 50, 686826), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 34, 42, 712382), 'autoMljar_total_time': datetime.timedelta(seconds=52, microseconds=25556), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 6, 14, 41, 383339), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 6, 19, 54, 471237), 'median_absolute_error_total_time': datetime.timedelta(seconds=313, microseconds=87898), 'r2_start_time': datetime.datetime(2022, 7, 11, 13, 5, 42, 854151), 'r2_end_time': datetime.datetime(2022, 7, 11, 13, 10, 55, 427713), 'r2_total_time': datetime.timedelta(seconds=312, microseconds=573562)}, '433.milc': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 35, 26, 655673), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 37, 28, 560253), 'nn_total_time': datetime.timedelta(seconds=121, microseconds=904580), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 34, 39, 500511), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 36, 48, 315084), 'autoKeras_total_time': datetime.timedelta(seconds=128, microseconds=814573), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 11, 23, 38, 540179), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 11, 28, 58, 105409), 'mean_absolute_error_total_time': datetime.timedelta(seconds=319, microseconds=565230), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 16, 45, 1, 986044), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 16, 50, 13, 816487), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=311, microseconds=830443), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 34, 42, 712397), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 35, 38, 977604), 'autoMljar_total_time': datetime.timedelta(seconds=56, microseconds=265207), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 6, 19, 54, 471259), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 6, 25, 6, 53399), 'median_absolute_error_total_time': datetime.timedelta(seconds=311, microseconds=582140), 'r2_start_time': datetime.datetime(2022, 7, 11, 13, 10, 55, 427733), 'r2_end_time': datetime.datetime(2022, 7, 11, 13, 16, 12, 494982), 'r2_total_time': datetime.timedelta(seconds=317, microseconds=67249)}, '434.zeusmp': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 37, 28, 975094), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 38, 45, 407095), 'nn_total_time': datetime.timedelta(seconds=76, microseconds=432001), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 36, 48, 427118), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 38, 58, 173687), 'autoKeras_total_time': datetime.timedelta(seconds=129, microseconds=746569), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 11, 28, 58, 105430), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 11, 34, 16, 510821), 'mean_absolute_error_total_time': datetime.timedelta(seconds=318, microseconds=405391), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 16, 50, 13, 816504), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 16, 55, 23, 751436), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=309, microseconds=934932), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 35, 38, 977620), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 36, 44, 877794), 'autoMljar_total_time': datetime.timedelta(seconds=65, microseconds=900174), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 6, 25, 6, 53420), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 6, 30, 16, 191289), 'median_absolute_error_total_time': datetime.timedelta(seconds=310, microseconds=137869), 'r2_start_time': datetime.datetime(2022, 7, 11, 13, 16, 12, 494996), 'r2_end_time': datetime.datetime(2022, 7, 11, 13, 21, 34, 523947), 'r2_total_time': datetime.timedelta(seconds=322, microseconds=28951)}, '435.gromacs': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 38, 45, 800517), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 39, 52, 873334), 'nn_total_time': datetime.timedelta(seconds=67, microseconds=72817), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 38, 58, 291568), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 41, 1, 754447), 'autoKeras_total_time': datetime.timedelta(seconds=123, microseconds=462879), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 11, 34, 16, 510840), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 11, 39, 38, 484757), 'mean_absolute_error_total_time': datetime.timedelta(seconds=321, microseconds=973917), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 16, 55, 23, 751451), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 17, 0, 35, 814775), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=312, microseconds=63324), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 36, 44, 877807), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 37, 39, 887974), 'autoMljar_total_time': datetime.timedelta(seconds=55, microseconds=10167), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 6, 30, 16, 191308), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 6, 35, 26, 703014), 'median_absolute_error_total_time': datetime.timedelta(seconds=310, microseconds=511706), 'r2_start_time': datetime.datetime(2022, 7, 11, 13, 21, 34, 523967), 'r2_end_time': datetime.datetime(2022, 7, 11, 13, 26, 56, 771456), 'r2_total_time': datetime.timedelta(seconds=322, microseconds=247489)}, '437.leslie3d': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 39, 53, 302089), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 40, 47, 323690), 'nn_total_time': datetime.timedelta(seconds=54, microseconds=21601), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 41, 1, 871758), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 43, 1, 230252), 'autoKeras_total_time': datetime.timedelta(seconds=119, microseconds=358494), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 11, 39, 38, 484773), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 11, 45, 3, 316868), 'mean_absolute_error_total_time': datetime.timedelta(seconds=324, microseconds=832095), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 17, 0, 35, 814792), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 17, 5, 46, 358107), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=310, microseconds=543315), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 37, 39, 887989), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 38, 42, 859235), 'autoMljar_total_time': datetime.timedelta(seconds=62, microseconds=971246), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 6, 35, 26, 703033), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 6, 40, 47, 790406), 'median_absolute_error_total_time': datetime.timedelta(seconds=321, microseconds=87373), 'r2_start_time': datetime.datetime(2022, 7, 11, 13, 26, 56, 771472), 'r2_end_time': datetime.datetime(2022, 7, 11, 13, 32, 13, 656246), 'r2_total_time': datetime.timedelta(seconds=316, microseconds=884774)}, '447.dealII': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 40, 47, 754285), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 41, 58, 5381), 'nn_total_time': datetime.timedelta(seconds=70, microseconds=251096), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 43, 1, 351063), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 45, 27, 107483), 'autoKeras_total_time': datetime.timedelta(seconds=145, microseconds=756420), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 11, 45, 3, 316912), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 11, 50, 23, 269911), 'mean_absolute_error_total_time': datetime.timedelta(seconds=319, microseconds=952999), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 17, 5, 46, 358126), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 17, 10, 56, 809556), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=310, microseconds=451430), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 38, 42, 859248), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 39, 44, 856282), 'autoMljar_total_time': datetime.timedelta(seconds=61, microseconds=997034), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 6, 40, 47, 790426), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 6, 45, 59, 401098), 'median_absolute_error_total_time': datetime.timedelta(seconds=311, microseconds=610672), 'r2_start_time': datetime.datetime(2022, 7, 11, 13, 32, 13, 656278), 'r2_end_time': datetime.datetime(2022, 7, 11, 13, 37, 28, 747329), 'r2_total_time': datetime.timedelta(seconds=315, microseconds=91051)}, '450.soplex': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 41, 58, 408433), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 44, 3, 336437), 'nn_total_time': datetime.timedelta(seconds=124, microseconds=928004), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 45, 27, 224292), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 47, 21, 750180), 'autoKeras_total_time': datetime.timedelta(seconds=114, microseconds=525888), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 11, 50, 23, 269929), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 11, 55, 43, 24651), 'mean_absolute_error_total_time': datetime.timedelta(seconds=319, microseconds=754722), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 17, 10, 56, 809574), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 17, 16, 7, 39383), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=310, microseconds=229809), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 39, 44, 856299), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 40, 53, 534594), 'autoMljar_total_time': datetime.timedelta(seconds=68, microseconds=678295), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 6, 45, 59, 401120), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 6, 51, 9, 935619), 'median_absolute_error_total_time': datetime.timedelta(seconds=310, microseconds=534499), 'r2_start_time': datetime.datetime(2022, 7, 11, 13, 37, 28, 747343), 'r2_end_time': datetime.datetime(2022, 7, 11, 13, 42, 42, 559924), 'r2_total_time': datetime.timedelta(seconds=313, microseconds=812581)}, '453.povray': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 44, 3, 742029), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 45, 13, 563011), 'nn_total_time': datetime.timedelta(seconds=69, microseconds=820982), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 47, 21, 864166), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 49, 20, 478265), 'autoKeras_total_time': datetime.timedelta(seconds=118, microseconds=614099), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 11, 55, 43, 24671), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 12, 1, 5, 713940), 'mean_absolute_error_total_time': datetime.timedelta(seconds=322, microseconds=689269), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 17, 16, 7, 39401), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 17, 21, 16, 555684), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=309, microseconds=516283), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 40, 53, 534608), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 42, 0, 616985), 'autoMljar_total_time': datetime.timedelta(seconds=67, microseconds=82377), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 6, 51, 9, 935638), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 6, 56, 21, 4469), 'median_absolute_error_total_time': datetime.timedelta(seconds=311, microseconds=68831), 'r2_start_time': datetime.datetime(2022, 7, 11, 13, 42, 42, 559943), 'r2_end_time': datetime.datetime(2022, 7, 11, 13, 47, 57, 539458), 'r2_total_time': datetime.timedelta(seconds=314, microseconds=979515)}, '454.calculix': {'nn_start_time': datetime.datetime(2022, 7, 10, 8, 45, 13, 959455), 'nn_end_time': datetime.datetime(2022, 7, 10, 8, 47, 23, 498195), 'nn_total_time': datetime.timedelta(seconds=129, microseconds=538740), 'autoKeras_start_time': datetime.datetime(2022, 7, 10, 9, 49, 20, 596244), 'autoKeras_end_time': datetime.datetime(2022, 7, 10, 9, 51, 18, 843765), 'autoKeras_total_time': datetime.timedelta(seconds=118, microseconds=247521), 'mean_absolute_error_start_time': datetime.datetime(2022, 7, 10, 12, 1, 5, 713960), 'mean_absolute_error_end_time': datetime.datetime(2022, 7, 10, 12, 6, 23, 177315), 'mean_absolute_error_total_time': datetime.timedelta(seconds=317, microseconds=463355), 'mean_squared_error_start_time': datetime.datetime(2022, 7, 10, 17, 21, 16, 555703), 'mean_squared_error_end_time': datetime.datetime(2022, 7, 10, 17, 26, 26, 502849), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=309, microseconds=947146), 'autoMljar_start_time': datetime.datetime(2022, 7, 10, 17, 42, 0, 616999), 'autoMljar_end_time': datetime.datetime(2022, 7, 10, 17, 43, 9, 80816), 'autoMljar_total_time': datetime.timedelta(seconds=68, microseconds=463817), 'median_absolute_error_start_time': datetime.datetime(2022, 7, 11, 6, 56, 21, 4487), 'median_absolute_error_end_time': datetime.datetime(2022, 7, 11, 7, 1, 31, 719198), 'median_absolute_error_total_time': datetime.timedelta(seconds=310, microseconds=714711), 'r2_start_time': datetime.datetime(2022, 7, 11, 13, 47, 57, 539475), 'r2_end_time': datetime.datetime(2022, 7, 11, 13, 53, 9, 171154), 'r2_total_time': datetime.timedelta(seconds=311, microseconds=631679)}}


#### algos

In [ ]:


for algo_idx, algo in enumerate(algo_list):
  # Perform train-test data split and scaling
  # perform_dataset_ops(benchmark_list[algo_idx], algo_list[algo_idx], algo_fname_list[algo_idx])

  # algos[algo].update({"nn_start_time":datetime.datetime.now()})
  # build_nn_model(algo)
  # algos[algo].update({"nn_end_time":datetime.datetime.now()})
  # algos[algo].update({"nn_total_time":algos[algo]['nn_end_time']-algos[algo]['nn_start_time']})


  # algos[algo].update({"autoKeras_start_time":datetime.datetime.now()})
  # build_autokeras(algo)
  # algos[algo].update({"autoKeras_end_time":datetime.datetime.now()})
  # algos[algo].update({"autoKeras_total_time":algos[algo]['autoKeras_end_time']-algos[algo]['autoKeras_start_time']})

  # mean_absolute_error
  # algos[algo].update({"mean_absolute_error_start_time":datetime.datetime.now()})
  # build_autoPytorch(algo,"mean_absolute_error")
  # algos[algo].update({"mean_absolute_error_end_time":datetime.datetime.now()})
  # algos[algo].update({"mean_absolute_error_total_time":algos[algo]['mean_absolute_error_end_time']-algos[algo]['mean_absolute_error_start_time']})

  # # mean_squared_error
  # algos[algo].update({"mean_squared_error_start_time":datetime.datetime.now()})
  # build_autoPytorch(algo,"mean_squared_error")
  # algos[algo].update({"mean_squared_error_end_time":datetime.datetime.now()})
  # algos[algo].update({"mean_squared_errorr_total_time":algos[algo]['mean_squared_error_end_time']-algos[algo]['mean_squared_error_start_time']})

  # # root_mean_squared_error 1 error: Memout (used more than 4096 MB). in float
  # algos[algo].update({"root_mean_squared_error_start_time":datetime.datetime.now()})
  # build_autoPytorch(algo,"root_mean_squared_error")
  # algos[algo].update({"root_mean_squared_error_end_time":datetime.datetime.now()})
  # algos[algo].update({"root_mean_squared_error_total_time":algos[algo]['root_mean_squared_error_end_time']-algos[algo]['root_mean_squared_error_start_time']})

  # # median_absolute_error 1
  # algos[algo].update({"median_absolute_error_start_time":datetime.datetime.now()})
  # build_autoPytorch(algo,"median_absolute_error")
  # algos[algo].update({"median_absolute_error_end_time":datetime.datetime.now()})
  # algos[algo].update({"median_absolute_error_total_time":algos[algo]['median_absolute_error_end_time']-algos[algo]['median_absolute_error_start_time']})

  # # r2 1
  algos[algo].update({"r2_start_time":datetime.datetime.now()})
  build_autoPytorch(algo,"r2")
  algos[algo].update({"r2_end_time":datetime.datetime.now()})
  algos[algo].update({"r2_total_time":algos[algo]['r2_end_time']-algos[algo]['r2_start_time']})
  
  # algos[algo].update({"autoMljar_start_time":datetime.datetime.now()})
  # build_mljar(algo)
  # algos[algo].update({"autoMljar_end_time":datetime.datetime.now()})
  # algos[algo].update({"autoMljar_total_time":algos[algo]['autoMljar_end_time']-algos[algo]['autoMljar_start_time']})

[WARNING] [2022-07-11 12:09:52,635:Client-Validation] AutoPyTorch previously received features of type <class 'numpy.ndarray'> yet the current features have type <class 'pandas.core.frame.DataFrame'>. Changing the dtype of inputs to an estimator might cause problems
[ERROR] [2022-07-11 12:09:56,882:Client-AutoPyTorch:5dfd24f6-0112-11ed-8178-0242ac1c0002:1] Traditional prediction for lgb failed with run state StatusType.CRASHED.
Additional info:
traceback: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/autoPyTorch/evaluation/tae.py", line 39, in fit_predict_try_except_decorator
    ta(queue=queue, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/autoPyTorch/evaluation/train_evaluator.py", line 485, in eval_function
    evaluator.fit_predict_and_loss()
  File "/usr/local/lib/python3.7/dist-packages/autoPyTorch/evaluation/train_evaluator.py", line 166, in fit_predict_and_loss
    add_pipeline_to_self=True)
  File "/usr/local/lib/python3.7/dist-pa

In [ ]:
print(algos) 

{'npb_ep': {'nn_start_time': datetime.datetime(2022, 6, 28, 8, 8, 59, 73052), 'nn_end_time': datetime.datetime(2022, 6, 28, 8, 9, 8, 643781), 'nn_total_time': datetime.timedelta(seconds=9, microseconds=570729), 'autoKeras_start_time': datetime.datetime(2022, 6, 28, 8, 12, 9, 740446), 'autoKeras_end_time': datetime.datetime(2022, 6, 28, 8, 13, 12, 629780), 'autoKeras_total_time': datetime.timedelta(seconds=62, microseconds=889334), 'mean_absolute_error_start_time': datetime.datetime(2022, 6, 28, 8, 26, 9, 327839), 'mean_absolute_error_end_time': datetime.datetime(2022, 6, 28, 8, 31, 26, 828293), 'mean_absolute_error_total_time': datetime.timedelta(seconds=317, microseconds=500454), 'mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 9, 3, 33, 761797), 'mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 9, 8, 53, 459305), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=319, microseconds=697508), 'root_mean_squared_error_start_time': datetime.datetime(20

In [ ]:
print(algos)

NameError: ignored

In [ ]:
save_dict_to_csv(algos,'comparisonAll_all_spec_int_10')

In [ ]:
tenp = {'600.perlbench_s': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 8, 49, 828632), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 14, 11, 743075), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=321, microseconds=914443)}, '602.gcc_s': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 14, 11, 743101), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 19, 31, 333734), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=319, microseconds=590633)}, '605.mcf_s': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 19, 31, 333754), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 24, 43, 716858), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=312, microseconds=383104)}, '620.omnetpp_s': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 24, 43, 716878), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 30, 2, 588857), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=318, microseconds=871979)}, '623.xalancbmk_s': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 30, 2, 588875), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 35, 19, 435745), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=316, microseconds=846870)}, '625.x264_s': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 35, 19, 435768), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 40, 39, 709389), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=320, microseconds=273621)}, '631.deepsjeng_s': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 40, 39, 709409), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 46, 3, 682559), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=323, microseconds=973150)}, '641.leela_s': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 46, 3, 682577), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 51, 24, 176185), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=320, microseconds=493608)}, '648.exchange2_s': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 51, 24, 176201), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 56, 45, 642040), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=321, microseconds=465839)}, '657.xz_s': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 56, 45, 642056), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 14, 1, 59, 23944), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=313, microseconds=381888)}, '401.bzip2': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 14, 1, 59, 23963), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 14, 7, 22, 864066), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=323, microseconds=840103)}, '456.hmmer': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 14, 7, 22, 864085), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 14, 12, 37, 362091), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=314, microseconds=498006)}, '458.sjeng': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 14, 12, 37, 362109), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 14, 17, 52, 542784), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=315, microseconds=180675)}, '462.libquantum': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 14, 17, 52, 542804), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 14, 23, 15, 907216), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=323, microseconds=364412)}, '473.astar': {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 14, 23, 15, 907234), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 14, 28, 38, 843503), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=322, microseconds=936269)}}

print(tenp.items())

dict_items([('600.perlbench_s', {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 8, 49, 828632), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 14, 11, 743075), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=321, microseconds=914443)}), ('602.gcc_s', {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 14, 11, 743101), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 19, 31, 333734), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=319, microseconds=590633)}), ('605.mcf_s', {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 19, 31, 333754), 'root_mean_squared_error_end_time': datetime.datetime(2022, 7, 7, 13, 24, 43, 716858), 'root_mean_squared_error_total_time': datetime.timedelta(seconds=312, microseconds=383104)}), ('620.omnetpp_s', {'root_mean_squared_error_start_time': datetime.datetime(2022, 7, 7, 13, 24, 43, 716878), 'root_mean_squared_e

In [ ]:
for key,val in tenp.items():
  if key not in algos:
    algos[key]=val
  else:
    for key2,val2  in val.items():
      if key2 not in algos[key]:
        algos[key][key2]=val2
      else:
        algos[key][key2]=val2

In [ ]:
print(algos)

{'npb_ep': {'nn_start_time': datetime.datetime(2022, 6, 28, 8, 8, 59, 73052), 'nn_end_time': datetime.datetime(2022, 6, 28, 8, 9, 8, 643781), 'nn_total_time': datetime.timedelta(seconds=9, microseconds=570729), 'autoKeras_start_time': datetime.datetime(2022, 6, 28, 8, 12, 9, 740446), 'autoKeras_end_time': datetime.datetime(2022, 6, 28, 8, 13, 12, 629780), 'autoKeras_total_time': datetime.timedelta(seconds=62, microseconds=889334), 'mean_absolute_error_start_time': datetime.datetime(2022, 6, 28, 8, 26, 9, 327839), 'mean_absolute_error_end_time': datetime.datetime(2022, 6, 28, 8, 31, 26, 828293), 'mean_absolute_error_total_time': datetime.timedelta(seconds=317, microseconds=500454), 'mean_squared_error_start_time': datetime.datetime(2022, 6, 28, 9, 3, 33, 761797), 'mean_squared_error_end_time': datetime.datetime(2022, 6, 28, 9, 8, 53, 459305), 'mean_squared_errorr_total_time': datetime.timedelta(seconds=319, microseconds=697508), 'root_mean_squared_error_start_time': datetime.datetime(20

# ploatting

In [37]:
#Plotting

#benchmark_list = ["NPB", "NPB", "SPEC2006", "SPEC2006"]
#algo_list = ["npb_ep", "npb_mg", "leslie3d", "perlbench"]

spec_float_benchmark_list = ["SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006"]
spec_float_algo_list = ["603.bwaves_s","607.cactuBSSN_s","619.lbm_s","621.wrf_s","627.cam4_s","628.pop2_s","638.imagick_s","644.nab_s","649.fotonik3d_s","654.roms_s","416.gamess","433.milc","434.zeusmp","435.gromacs","437.leslie3d","447.dealII","450.soplex","453.povray","454.calculix"]
spec_float_algo_fname_list = ["603.bwaves_s","607.cactuBSSN_s","619.lbm_s","621.wrf_s","627.cam4_s","628.pop2_s","638.imagick_s","644.nab_s","649.fotonik3d_s","654.roms_s","416.gamess","433.milc","434.zeusmp","435.gromacs","437.leslie3d","447.dealII","450.soplex","453.povray","454.calculix"]
spec_float_benchmark_title = 'SPEC_Float'

spec_int_benchmark_list = ["SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006"]
spec_int_algo_list = ["600.perlbench_s","602.gcc_s","605.mcf_s","620.omnetpp_s","623.xalancbmk_s","625.x264_s","631.deepsjeng_s","641.leela_s","648.exchange2_s","657.xz_s","401.bzip2","456.hmmer","458.sjeng","462.libquantum","473.astar"]
spec_int_algo_fname_list = ["600.perlbench_s","602.gcc_s","605.mcf_s","620.omnetpp_s","623.xalancbmk_s","625.x264_s","631.deepsjeng_s","641.leela_s","648.exchange2_s","657.xz_s","401.bzip2","456.hmmer","458.sjeng","462.libquantum","473.astar"]
spec_int_benchmark_title = 'SPEC_Int'

npb_other_benchmark_list = ["NPB","NPB","NPB","NPB","OTHER","OTHER","OTHER"]
npb_other_algo_list = ["npb_ep","npb_mg","npb_sp","npb_sp-mz","matmul","montecarlo","quicksort"]
npb_other_algo_fname_list = ["npb_ep","npb_mg","npb_sp","npb_sp-mz","matmul","montecarlo","quicksort"]
npb_other_benchmark_title = 'NPB_OTHER'




In [38]:
# Assign variable with which list is to be processed
benchmark_list = spec_float_benchmark_list
algo_list = spec_float_algo_list
algo_fname_list = spec_float_algo_fname_list
benchmark_title = spec_float_benchmark_title
plot_file_ext=".eps"

In [45]:
# Assign variable with which list is to be processed
benchmark_list = spec_int_benchmark_list
algo_list = spec_int_algo_list
algo_fname_list = spec_int_algo_fname_list
benchmark_title = spec_int_benchmark_title
plot_file_ext=".eps"

In [53]:
# Assign variable with which list is to be processed
benchmark_list = npb_other_benchmark_list
algo_list = npb_other_algo_list
algo_fname_list = npb_other_algo_fname_list
benchmark_title = npb_other_benchmark_title
plot_file_ext=".eps"

# plotting

## nbp

In [54]:
drive_location = r"/content/drive/MyDrive/Colab Notebooks/Internship/AllAlgos/"

### pytorch

In [55]:

set_pub()
plot_all_pytorch(benchmark_title, algo_list)
#plot_nnmodel_vs_autokeras(benchmark_title, algo_list)

# plot_iteration_metric(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

### nnmodel vs auto keras

In [56]:
plot_nnmodel_vs_autokeras(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

### nnmodel vs auto keras vs auto pytorch

In [57]:
plot_nnmodel_vs_autokeras_autoPytorch(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

### nnmodel vs autokeras vs auto pytorch vs mljar

In [58]:
plot_nnmodel_vs_autokeras_autoPytorch_mljar(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

In [59]:
plot_nnmodel_vs_autokeras_autoPytorch_mljar(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

## spec int

In [46]:
drive_location = r"/content/drive/MyDrive/Colab Notebooks/Internship/SpecInt/"

### pytorch

In [47]:

set_pub()
plot_all_pytorch(benchmark_title, algo_list)
#plot_nnmodel_vs_autokeras(benchmark_title, algo_list)

# plot_iteration_metric(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

### nnmodel vs auto keras

In [48]:
plot_nnmodel_vs_autokeras(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

### nnmodel vs auto keras vs auto pytorch

In [49]:
plot_nnmodel_vs_autokeras_autoPytorch(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

### nnmodel vs autokeras vs auto pytorch vs mljar

In [51]:
plot_nnmodel_vs_autokeras_autoPytorch_mljar(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

In [52]:
plot_nnmodel_vs_autokeras_autoPytorch_mljar(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

## spec float

In [39]:
drive_location = r"/content/drive/MyDrive/Colab Notebooks/Internship/SpecFloat/"

### pytorch

In [40]:

set_pub()
plot_all_pytorch1(benchmark_title, algo_list)
#plot_nnmodel_vs_autokeras(benchmark_title, algo_list)

# plot_iteration_metric(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

### nnmodel vs auto keras

In [41]:
plot_nnmodel_vs_autokeras(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

### nnmodel vs auto keras vs auto pytorch

In [42]:
plot_nnmodel_vs_autokeras_autoPytorch(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

### nnmodel vs autokeras vs auto pytorch vs mljar

In [43]:
plot_nnmodel_vs_autokeras_autoPytorch_mljar(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.

In [44]:
plot_nnmodel_vs_autokeras_autoPytorch_mljar(benchmark_title, algo_list)

Output hidden; open in https://colab.research.google.com to view.